In [5]:
# --- 1. 安装依赖 ---
!pip install xarray netCDF4 matplotlib geopandas rasterio rioxarray --quiet

# --- 2. 挂载Google Drive ---
from google.colab import drive
drive.mount('/content/drive')

# --- 3. 导入库 ---
import os
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 1. Verification: NO2 and SO2 File Structure Check

In [ ]:
# Complete Verification: NO2 and SO2 File Structure Check
import os, numpy as np, glob

def check_file_structures():
    """Complete verification of NO2 and SO2 file structures"""
    print("🔍 Complete File Structure Verification")
    print("="*60)

    base_path = "/content/drive/MyDrive/Feature_Stacks"

    # Check NO2 structure
    print("\n📁 Checking NO2 File Structure:")
    print("-" * 40)

    no2_dir = os.path.join(base_path, "NO2_2019")
    if not os.path.exists(no2_dir):
        print(f"❌ NO2 directory not found: {no2_dir}")
        return

    no2_files = sorted(glob.glob(os.path.join(no2_dir, "NO2_stack_*.npz")))
    if len(no2_files) == 0:
        print(f"❌ No NO2 files found in: {no2_dir}")
        return

    print(f"✅ Found {len(no2_files)} NO2 files")

    # Check first NO2 file
    no2_file = no2_files[0]
    print(f"📄 Analyzing: {os.path.basename(no2_file)}")

    try:
        with np.load(no2_file, allow_pickle=True) as data:
            print(f" NO2 file keys: {list(data.files)}")

            # Check for X matrix
            if 'X' in data.files:
                X = data['X']
                print(f"✅ NO2 has X matrix: shape={X.shape}")
            else:
                print(f"❌ NO2 missing X matrix")
                print(f"   Available keys: {[k for k in data.files if k not in ['coverage_rate', 'valid_pixels', 'total_pixels', 'coverage_in_aoi', 'aoi_pixels', 'trainable']]}")

            # Check for target and mask
            if 'no2_target' in data.files:
                target = data['no2_target']
                print(f"✅ NO2 has target: shape={target.shape}")
            else:
                print(f"❌ NO2 missing target")

            if 'no2_mask' in data.files:
                mask = data['no2_mask']
                print(f"✅ NO2 has mask: shape={mask.shape}")
            else:
                print(f"❌ NO2 missing mask")

    except Exception as e:
        print(f"❌ Error loading NO2 file: {e}")

    # Check SO2 structure
    print("\n📁 Checking SO2 File Structure:")
    print("-" * 40)

    so2_dir = os.path.join(base_path, "SO2_2019")
    if not os.path.exists(so2_dir):
        print(f"❌ SO2 directory not found: {so2_dir}")
        return

    so2_files = sorted(glob.glob(os.path.join(so2_dir, "SO2_stack_*.npz")))
    if len(so2_files) == 0:
        print(f"❌ No SO2 files found in: {so2_dir}")
        return

    print(f"✅ Found {len(so2_files)} SO2 files")

    # Check first SO2 file
    so2_file = so2_files[0]
    print(f"📄 Analyzing: {os.path.basename(so2_file)}")

    try:
        with np.load(so2_file, allow_pickle=True) as data:
            print(f" SO2 file keys: {list(data.files)}")

            # Check for X matrix
            if 'X' in data.files:
                X = data['X']
                print(f"✅ SO2 has X matrix: shape={X.shape}")
            else:
                print(f"❌ SO2 missing X matrix")

            # Check for target and mask
            if 'y' in data.files:
                target = data['y']
                print(f"✅ SO2 has target: shape={target.shape}")
            else:
                print(f"❌ SO2 missing target")

            if 'mask' in data.files:
                mask = data['mask']
                print(f"✅ SO2 has mask: shape={mask.shape}")
            else:
                print(f"❌ SO2 missing mask")

    except Exception as e:
        print(f"❌ Error loading SO2 file: {e}")

    # Summary
    print("\n📋 Summary:")
    print("-" * 40)
    print("File structure comparison:")
    print("  NO2: Dictionary structure (no X matrix)")
    print("  SO2: Matrix structure (has X matrix)")
    print("\nRecommendation:")
    print("  Need to modify NO2 scaler to handle dictionary structure")

def check_no2_feature_availability():
    """Check if NO2 has all required features for scaler"""
    print("\n🔍 Checking NO2 Feature Availability:")
    print("-" * 40)

    base_path = "/content/drive/MyDrive/Feature_Stacks"
    no2_dir = os.path.join(base_path, "NO2_2019")
    no2_files = sorted(glob.glob(os.path.join(no2_dir, "NO2_stack_*.npz")))

    if len(no2_files) == 0:
        print("❌ No NO2 files found")
        return

    # Required features for scaler
    required_features = [
        'u10', 'v10', 'blh', 'tp', 't2m', 'sp', 'str', 'ssr_clr',
        'no2_lag_1day', 'no2_neighbor',
        'dem', 'slope', 'pop'
    ]

    try:
        with np.load(no2_files[0], allow_pickle=True) as data:
            available_keys = list(data.files)
            print(f"📊 Available features: {len(available_keys)}")

            missing_features = []
            available_features = []

            for feature in required_features:
                if feature in available_keys:
                    available_features.append(feature)
                else:
                    missing_features.append(feature)

            print(f"✅ Available required features: {len(available_features)}")
            print(f"   {available_features}")

            if missing_features:
                print(f"❌ Missing required features: {len(missing_features)}")
                print(f"   {missing_features}")
            else:
                print("✅ All required features available for scaler")

    except Exception as e:
        print(f"❌ Error checking features: {e}")

def suggest_no2_scaler_fix():
    """Suggest how to fix NO2 scaler for dictionary structure"""
    print("\n🛠️ Suggested NO2 Scaler Fix:")
    print("-" * 40)

    print("Current NO2 scaler expects:")
    print("  - data['X'] (matrix structure)")
    print("  - data['mask'] (unified mask)")

    print("\nActual NO2 structure:")
    print("  - Dictionary of individual features")
    print("  - data['no2_mask'] (specific mask)")

    print("\nRequired changes:")
    print("1. Build X matrix from dictionary features")
    print("2. Use data['no2_mask'] instead of data['mask']")
    print("3. Handle feature ordering manually")

    print("\nCode changes needed:")
    print("""
    # Instead of:
    X = data['X']
    valid_mask = data['mask'] == 1

    # Use:
    feature_order = ['dem', 'slope', 'pop', 'u10', 'v10', ...]
    X = np.stack([data[name] for name in feature_order], axis=0)
    valid_mask = data['no2_mask'] == 1
    """)

# Run all checks
print(" Starting Complete File Structure Verification")
print("="*80)

check_file_structures()
check_no2_feature_availability()
suggest_no2_scaler_fix()

print("\n Next Steps:")
print("1. If NO2 missing X matrix → Modify NO2 scaler code")
print("2. If NO2 has X matrix → Run scaler directly")
print("3. Ensure mask semantics are correct (1=valid, 0=invalid)")

 Starting Complete File Structure Verification
🔍 Complete File Structure Verification

📁 Checking NO2 File Structure:
----------------------------------------
✅ Found 365 NO2 files
📄 Analyzing: NO2_stack_20190101.npz
 NO2 file keys: ['no2_target', 'no2_mask', 'dem', 'slope', 'pop', 'lulc_class_0', 'lulc_class_1', 'lulc_class_2', 'lulc_class_3', 'lulc_class_4', 'lulc_class_5', 'lulc_class_6', 'lulc_class_7', 'lulc_class_8', 'lulc_class_9', 'sin_doy', 'cos_doy', 'weekday_weight', 'u10', 'v10', 'blh', 'tp', 't2m', 'sp', 'str', 'ssr_clr', 'ws', 'wd_sin', 'wd_cos', 'no2_lag_1day', 'no2_neighbor', 'coverage_rate', 'valid_pixels', 'total_pixels', 'coverage_in_aoi', 'aoi_pixels', 'trainable']
❌ NO2 missing X matrix
   Available keys: ['no2_target', 'no2_mask', 'dem', 'slope', 'pop', 'lulc_class_0', 'lulc_class_1', 'lulc_class_2', 'lulc_class_3', 'lulc_class_4', 'lulc_class_5', 'lulc_class_6', 'lulc_class_7', 'lulc_class_8', 'lulc_class_9', 'sin_doy', 'cos_doy', 'weekday_weight', 'u10', 'v10', 

# 2. Generate NO2 standardization parameters

In [ ]:
# Generate NO2 standardization parameters (FINAL VERIFIED VERSION)
import os, numpy as np
from collections import defaultdict

# Create output directory
os.makedirs("/content/drive/MyDrive/Scalers", exist_ok=True)

BASE_NO2 = "/content/drive/MyDrive/Feature_Stacks"
OUT_DIR = "/content/drive/MyDrive/Scalers"

# Continuous features that need standardization (NO2 version)
CONTINUOUS_FEATURES = [
    'u10', 'v10', 'blh', 'tp', 't2m', 'sp', 'str', 'ssr_clr',
    'no2_lag_1day', 'no2_neighbor',  # NO2 related features
    'dem', 'slope', 'pop',
    'ws'  # Wind speed (if not already standardized)
]

# Features that should not be standardized
NON_STANDARDIZED = ['sin_doy', 'cos_doy', 'weekday_weight', 'wd_sin', 'wd_cos']

# Categorical features (one-hot encoded)
CATEGORICAL_FEATURES = [
    'lulc_class_0', 'lulc_class_1', 'lulc_class_2', 'lulc_class_3', 'lulc_class_4',
    'lulc_class_5', 'lulc_class_6', 'lulc_class_7', 'lulc_class_8', 'lulc_class_9'
]

def compute_no2_scalers(years=[2019, 2020, 2021], sample_per_file=1000):
    """Compute standardization parameters for NO2"""
    print("🚀 Starting NO2 standardization parameter computation...")
    print(f"Data path: {BASE_NO2}")
    print(f"Training years: {years}")
    print(f"Samples per file: {sample_per_file}")

    # Accumulate statistics
    stats = defaultdict(lambda: {'sum': 0.0, 'sumsq': 0.0, 'count': 0})
    feature_order = None

    total_files = 0
    processed_files = 0

    for year in years:
        year_dir = os.path.join(BASE_NO2, f"NO2_{year}")
        if not os.path.exists(year_dir):
            print(f"⚠️ Year directory not found: {year_dir}")
            continue

        files = [f for f in os.listdir(year_dir) if f.endswith('.npz')]
        total_files += len(files)
        print(f"\n Processing {year}: {len(files)} files")

        for i, fname in enumerate(files):
            if i % 50 == 0:
                print(f"  {i}/{len(files)} ({i/len(files)*100:.1f}%)")

            try:
                with np.load(os.path.join(year_dir, fname)) as data:
                    # NO2 uses dictionary structure, need to build X matrix
                    if feature_order is None:
                        # Define feature order for NO2 (based on verification results)
                        feature_order = [
                            'dem', 'slope', 'pop',
                            'lulc_class_0', 'lulc_class_1', 'lulc_class_2', 'lulc_class_3', 'lulc_class_4',
                            'lulc_class_5', 'lulc_class_6', 'lulc_class_7', 'lulc_class_8', 'lulc_class_9',
                            'sin_doy', 'cos_doy', 'weekday_weight',
                            'u10', 'v10', 'blh', 'tp', 't2m', 'sp', 'str', 'ssr_clr',
                            'ws', 'wd_sin', 'wd_cos', 'no2_lag_1day', 'no2_neighbor'
                        ]

                    # Build X matrix from dictionary features with robustness checks
                    # Strategy: Fill missing non-critical features with appropriate defaults
                    X_list = []
                    missing_critical = False

                    for name in feature_order:
                        if name in data.files:
                            X_list.append(data[name].astype(np.float32))
                        else:
                            print(f"   ⚠️ Missing feature: {name} in {fname}")
                            if name in CONTINUOUS_FEATURES:
                                # Critical feature missing - skip this file
                                print(f"   ❌ Critical continuous feature missing, skipping file")
                                missing_critical = True
                                break
                            else:
                                # Non-critical feature missing - fill with appropriate default
                                if name.startswith('lulc_class_'):
                                    fill_value = 0.0  # One-hot encoded, 0 is appropriate for missing class
                                elif name in ['sin_doy', 'cos_doy']:
                                    fill_value = 0.0  # Default to 0 for missing temporal features
                                elif name == 'weekday_weight':
                                    fill_value = 1.0  # Default to weekday weight
                                elif name in ['wd_sin', 'wd_cos']:
                                    fill_value = 0.0  # Default wind direction components
                                else:
                                    fill_value = 0.0  # Default fallback

                                print(f"   ℹ️ Filling missing non-critical feature {name} with {fill_value}")
                                # Get dimensions from first available feature
                                if X_list:
                                    H, W = X_list[0].shape
                                else:
                                    # Fallback dimensions (should not happen in practice)
                                    H, W = 300, 621
                                X_list.append(np.full((H, W), fill_value, dtype=np.float32))

                    if not missing_critical:
                        # All features handled (either available or filled), build X matrix
                        X = np.stack(X_list, axis=0).astype(np.float32)  # (C, H, W)

                        # CONFIRMED: NO2: mask==1 is valid pixels, mask==0 is invalid
                        # Additional robustness: ensure target > 0 for physical validity
                        valid_mask = (data['no2_mask'] == 1) & (data['no2_target'] > 0)

                        # --- 关键修正：正确处理2D掩膜索引 ---
                        C, H, W = X.shape
                        X_flat = X.reshape(C, -1)  # (C, H*W) - 打平空间维度
                        valid_flat = valid_mask.ravel()  # (H*W,) - 打平掩膜
                        valid_pixels = X_flat[:, valid_flat]  # (C, N_valid) - 正确提取有效像素

                        if valid_pixels.shape[1] == 0:
                            continue

                        # Random sampling to avoid memory issues
                        n_samples = min(sample_per_file, valid_pixels.shape[1])
                        if n_samples < valid_pixels.shape[1]:
                            indices = np.random.choice(valid_pixels.shape[1], n_samples, replace=False)
                            sampled = valid_pixels[:, indices]
                        else:
                            sampled = valid_pixels

                        # Filter NaN/inf
                        finite_mask = np.isfinite(sampled)

                        # Accumulate statistics
                        for j, feat_name in enumerate(feature_order):
                            if feat_name in CONTINUOUS_FEATURES:
                                values = sampled[j]
                                finite_values = values[finite_mask[j]]

                                if len(finite_values) > 0:
                                    # 1-99 percentile winsorization
                                    q1, q99 = np.percentile(finite_values, [1, 99])
                                    clipped = np.clip(finite_values, q1, q99)

                                    stats[feat_name]['sum'] += np.sum(clipped)
                                    stats[feat_name]['sumsq'] += np.sum(clipped**2)
                                    stats[feat_name]['count'] += len(clipped)
                            elif feat_name in CATEGORICAL_FEATURES:
                                # Categorical features don't need standardization
                                pass
                            elif feat_name in NON_STANDARDIZED:
                                # Non-standardized features don't need standardization
                                pass

                        processed_files += 1

            except Exception as e:
                print(f"❌ Failed to load file {fname}: {e}")
                continue

    print(f"\n📊 Processing statistics:")
    print(f"Total files: {total_files}")
    print(f"Successfully processed: {processed_files}")
    print(f"Processing rate: {processed_files/total_files*100:.1f}%")

    if not stats:
        print("❌ No valid samples found")
        return None

    # Compute final statistics
    scalers = {}
    print(f"\n🔢 Computing standardization parameters...")

    for feat_name in feature_order:
        if feat_name in CONTINUOUS_FEATURES:
            # 连续特征：统一处理，即使没有统计到也给默认值
            if feat_name in stats:
                count = stats[feat_name]['count']
                if count > 0:
                    mean = stats[feat_name]['sum'] / count
                    variance = (stats[feat_name]['sumsq'] / count) - (mean**2)
                    std = np.sqrt(max(variance, 1e-8))  # Avoid division by zero

                    scalers[feat_name] = {
                        'mean': float(mean),
                        'std': float(std),
                        'count': int(count)
                    }
                    print(f"  {feat_name}: mean={mean:.4f}, std={std:.4f}, count={count}")
                else:
                    # 有统计但count=0，给默认值
                    scalers[feat_name] = {'mean': 0.0, 'std': 1.0, 'count': 0}
                    print(f"  {feat_name}: No valid samples, using default values")
            else:
                # 完全没有统计到，也给默认值
                scalers[feat_name] = {'mean': 0.0, 'std': 1.0, 'count': 0}
                print(f"  {feat_name}: No statistics collected, using default values")
        elif feat_name in CATEGORICAL_FEATURES:
            scalers[feat_name] = {'type': 'categorical'}
            print(f"  {feat_name}: categorical (one-hot)")
        elif feat_name in NON_STANDARDIZED:
            scalers[feat_name] = {'type': 'non_standardized'}
            print(f"  {feat_name}: non_standardized")
        else:
            # 这种情况理论上不应该发生，因为feature_order应该包含所有特征
            scalers[feat_name] = {'type': 'unexpected'}
            print(f"  {feat_name}: unexpected type - check feature_order definition")

    return scalers, feature_order

def apply_no2_scaler(X: np.ndarray, feature_names: list, scaler_file: str = None):
    """
    Apply NO2 standardization to feature matrix

    Args:
        X: Feature matrix (C, H, W) or (C, N)
        feature_names: List of feature names
        scaler_file: Path to scaler file (if None, auto-detect)

    Returns:
        X_scaled: Standardized feature matrix
    """
    if scaler_file is None:
        scaler_file = os.path.join(OUT_DIR, "no2_scalers_2019_2021.npz")

    if not os.path.exists(scaler_file):
        raise FileNotFoundError(f"NO2 scaler file not found: {scaler_file}")

    with np.load(scaler_file, allow_pickle=True) as data:
        scalers = data['scalers'].item()

    X_scaled = X.copy().astype(np.float32)

    for i, feat_name in enumerate(feature_names):
        if feat_name in scalers and 'mean' in scalers[feat_name]:
            # Apply z-score normalization for continuous features
            mean = scalers[feat_name]['mean']
            std = scalers[feat_name]['std']
            X_scaled[i] = (X[i] - mean) / std
        # Skip categorical and non-standardized features (they remain unchanged)

    return X_scaled

# Compute NO2 standardization parameters
print("=" * 60)
print(" NO2 Standardization Parameter Computation (FINAL VERIFIED)")
print("=" * 60)

no2_scalers, no2_features = compute_no2_scalers()

if no2_scalers is not None:
    # Save results
    output_file = os.path.join(OUT_DIR, "no2_scalers_2019_2021.npz")
    np.savez_compressed(
        output_file,
        scalers=no2_scalers,
        feature_order=no2_features,
        metadata={
            'data_type': 'no2',
            'years': [2019, 2020, 2021],
            'mask_logic': 'mask==1 is valid, mask==0 is invalid (CONFIRMED FROM SOURCE CODE)',
            'total_features': len(no2_features),
            'continuous_features': len([f for f in no2_features if f in CONTINUOUS_FEATURES])
        }
    )

    print(f"\n✅ NO2 standardization parameters saved to: {output_file}")
    print(f" Feature statistics:")
    print(f"  Total features: {len(no2_features)}")
    print(f"  Continuous features: {len([f for f in no2_features if f in CONTINUOUS_FEATURES])}")
    print(f"  Categorical features: {len([f for f in no2_features if f in CATEGORICAL_FEATURES])}")
    print(f"  Non-standardized features: {len([f for f in no2_features if f in NON_STANDARDIZED])}")
    # Show parameters for first few continuous features
    print(f"\n🔍 Standardization parameters for first 5 continuous features:")
    continuous_count = 0
    for feat_name in no2_features:
        if feat_name in CONTINUOUS_FEATURES and feat_name in no2_scalers:
            if 'mean' in no2_scalers[feat_name]:
                print(f"  {feat_name}: mean={no2_scalers[feat_name]['mean']:.4f}, std={no2_scalers[feat_name]['std']:.4f}")
                continuous_count += 1
                if continuous_count >= 5:
                    break
else:
    print("❌ Computation failed")

print("\n Next step: Ready to train NO2 model!")

 NO2 Standardization Parameter Computation (FINAL VERIFIED)
🚀 Starting NO2 standardization parameter computation...
Data path: /content/drive/MyDrive/Feature_Stacks
Training years: [2019, 2020, 2021]
Samples per file: 1000

 Processing 2019: 365 files
  0/365 (0.0%)
  50/365 (13.7%)
  100/365 (27.4%)
  150/365 (41.1%)
  200/365 (54.8%)
  250/365 (68.5%)
  300/365 (82.2%)
  350/365 (95.9%)

 Processing 2020: 366 files
  0/366 (0.0%)
  50/366 (13.7%)
  100/366 (27.3%)
  150/366 (41.0%)
  200/366 (54.6%)
  250/366 (68.3%)
  300/366 (82.0%)
  350/366 (95.6%)

 Processing 2021: 365 files
  0/365 (0.0%)
  50/365 (13.7%)
  100/365 (27.4%)
  150/365 (41.1%)
  200/365 (54.8%)
  250/365 (68.5%)
  300/365 (82.2%)
  350/365 (95.9%)

📊 Processing statistics:
Total files: 1096
Successfully processed: 1071
Processing rate: 97.7%

🔢 Computing standardization parameters...
  dem: mean=606.2273, std=771.7322, count=1052695
  slope: mean=11.3112, std=11.0288, count=994379
  pop: mean=135.9125, std=369.51

In [ ]:
# Quick Check NO2 Scaler Results
import os
import numpy as np

def quick_check_no2_scaler_results():
    """快速检查NO2 scaler结果"""
    print("🔍 Quick NO2 Scaler Results Check")
    print("=" * 40)

    # 1. 检查文件是否存在
    scaler_file = "/content/drive/MyDrive/Scalers/no2_scalers_2019_2021.npz"

    if not os.path.exists(scaler_file):
        print(f"❌ Scaler file not found: {scaler_file}")
        return False

    print(f"✅ Scaler file exists")

    # 2. 加载并检查基本内容
    try:
        with np.load(scaler_file, allow_pickle=True) as data:
            scalers = data['scalers'].item()
            feature_order = data['feature_order']
            metadata = data['metadata'].item()

        print(f"✅ File loaded successfully")
        print(f"📊 Total features: {len(feature_order)}")
        print(f"📊 Years: {metadata.get('years', 'unknown')}")

    except Exception as e:
        print(f"❌ Failed to load: {e}")
        return False

    # 3. 检查关键连续特征
    expected_continuous = [
        'dem', 'slope', 'pop', 'u10', 'v10', 'blh', 'tp', 't2m', 'sp', 'str', 'ssr_clr',
        'ws', 'no2_lag_1day', 'no2_neighbor'
    ]

    print(f"\n🎯 Key Continuous Features Analysis:")
    continuous_ok = 0
    low_sample_features = []
    zero_value_features = []

    for feat in expected_continuous:
        if feat in scalers and 'mean' in scalers[feat]:
            count = scalers[feat]['count']
            mean = scalers[feat]['mean']
            std = scalers[feat]['std']

            if count > 0:
                print(f"  ✅ {feat}: {count:,} samples, mean={mean:.4f}, std={std:.4f}")
                continuous_ok += 1

                # 检查异常情况
                if count < 100000:  # 样本数太少
                    low_sample_features.append((feat, count))
                if abs(mean) < 0.001 and std < 0.001:  # 值几乎为0
                    zero_value_features.append((feat, mean, std))
            else:
                print(f"  ⚠️ {feat}: default values (no samples)")
        else:
            print(f"  ❌ {feat}: missing")

    # 4. 检查其他特征类型
    categorical_count = sum(1 for feat in feature_order
                           if feat in scalers and scalers[feat].get('type') == 'categorical')
    non_std_count = sum(1 for feat in feature_order
                       if feat in scalers and scalers[feat].get('type') == 'non_standardized')

    print(f"\n📊 Feature Type Summary:")
    print(f"  Continuous features: {continuous_ok}/14")
    print(f"  Categorical features: {categorical_count}")
    print(f"  Non-standardized features: {non_std_count}")

    # 5. 异常情况报告
    if low_sample_features:
        print(f"\n⚠️ Features with low sample counts:")
        for feat, count in low_sample_features:
            print(f"  {feat}: {count:,} samples")

    if zero_value_features:
        print(f"\n⚠️ Features with near-zero values:")
        for feat, mean, std in zero_value_features:
            print(f"  {feat}: mean={mean:.6f}, std={std:.6f}")

    # 6. 总体评估
    success_rate = (continuous_ok / 14) * 100
    print(f"\n🎯 Success Rate: {success_rate:.1f}%")

    if success_rate >= 80:
        print(f"🎉 NO2 Scaler looks good!")
        if low_sample_features or zero_value_features:
            print(f"💡 Note: Some features have unusual values, but this might be normal for the dataset")
        return True
    else:
        print(f"⚠️ NO2 Scaler needs attention")
        return False

if __name__ == "__main__":
    quick_check_no2_scaler_results()


🔍 Quick NO2 Scaler Results Check
✅ Scaler file exists
✅ File loaded successfully
📊 Total features: 29
📊 Years: [2019, 2020, 2021]

🎯 Key Continuous Features Analysis:
  ✅ dem: 1,052,695 samples, mean=606.2273, std=771.7322
  ✅ slope: 994,379 samples, mean=11.3112, std=11.0288
  ✅ pop: 992,667 samples, mean=135.9125, std=369.5149
  ✅ u10: 1,052,695 samples, mean=-0.2344, std=1.5692
  ✅ v10: 1,052,695 samples, mean=0.1392, std=1.4423
  ✅ blh: 1,052,695 samples, mean=951.2973, std=558.4578
  ✅ tp: 1,052,695 samples, mean=0.0003, std=0.0009
  ✅ t2m: 1,052,695 samples, mean=15.8182, std=9.8316
  ⚠️ sp: default values (no samples)
  ✅ str: 1,631 samples, mean=-29437.2015, std=32870.4722
  ⚠️ ssr_clr: default values (no samples)
  ✅ ws: 1,052,695 samples, mean=1.7724, std=1.2206
  ✅ no2_lag_1day: 1,033,884 samples, mean=0.0000, std=0.0001
  ✅ no2_neighbor: 1,052,695 samples, mean=0.0001, std=0.0001

📊 Feature Type Summary:
  Continuous features: 12/14
  Categorical features: 10
  Non-standard

In [ ]:
# Validate NO2 Scaler Results
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

def validate_no2_scaler_results():
    """验证NO2 scaler运行结果"""
    print("🔍 NO2 Scaler Results Validation")
    print("=" * 50)

    # 1. 检查输出文件是否存在
    scaler_file = "/content/drive/MyDrive/Scalers/no2_scalers_2019_2021.npz"

    if not os.path.exists(scaler_file):
        print(f"❌ Scaler file not found: {scaler_file}")
        return False

    print(f"✅ Scaler file found: {scaler_file}")

    # 2. 加载并检查scaler内容
    try:
        with np.load(scaler_file, allow_pickle=True) as data:
            scalers = data['scalers'].item()
            feature_order = data['feature_order']
            metadata = data['metadata'].item()

        print(f"✅ Successfully loaded scaler data")

    except Exception as e:
        print(f"❌ Failed to load scaler file: {e}")
        return False

    # 3. 检查元数据
    print(f"\n📊 Metadata:")
    print(f"  Data type: {metadata.get('data_type', 'unknown')}")
    print(f"  Years: {metadata.get('years', 'unknown')}")
    print(f"  Mask logic: {metadata.get('mask_logic', 'unknown')}")
    print(f"  Total features: {metadata.get('total_features', 'unknown')}")
    print(f"  Continuous features: {metadata.get('continuous_features', 'unknown')}")

    # 4. 检查特征顺序
    print(f"\n📋 Feature Order ({len(feature_order)} features):")
    for i, feat in enumerate(feature_order):
        print(f"  {i+1:2d}. {feat}")

    # 5. 检查scaler参数
    print(f"\n🔢 Scaler Parameters:")

    continuous_count = 0
    categorical_count = 0
    non_standardized_count = 0
    default_count = 0

    for feat_name in feature_order:
        if feat_name in scalers:
            scaler_info = scalers[feat_name]

            if 'mean' in scaler_info:
                # 连续特征
                mean = scaler_info['mean']
                std = scaler_info['std']
                count = scaler_info['count']
                continuous_count += 1

                if count > 0:
                    print(f"  ✅ {feat_name}: mean={mean:.4f}, std={std:.4f}, count={count:,}")
                else:
                    print(f"  ⚠️ {feat_name}: default values (mean=0, std=1, count=0)")
                    default_count += 1

            elif scaler_info.get('type') == 'categorical':
                categorical_count += 1
                print(f"  📊 {feat_name}: categorical (one-hot)")

            elif scaler_info.get('type') == 'non_standardized':
                non_standardized_count += 1
                print(f"  🚫 {feat_name}: non_standardized")

            else:
                print(f"  ❓ {feat_name}: {scaler_info}")

    # 6. 统计总结
    print(f"\n📈 Summary Statistics:")
    print(f"  Total features: {len(feature_order)}")
    print(f"  Continuous features: {continuous_count}")
    print(f"  Categorical features: {categorical_count}")
    print(f"  Non-standardized features: {non_standardized_count}")
    print(f"  Features with default values: {default_count}")

    # 7. 验证关键连续特征
    expected_continuous = [
        'dem', 'slope', 'pop', 'u10', 'v10', 'blh', 'tp', 't2m', 'sp', 'str', 'ssr_clr',
        'ws', 'no2_lag_1day', 'no2_neighbor'
    ]

    print(f"\n🎯 Key Continuous Features Validation:")
    missing_features = []
    for feat in expected_continuous:
        if feat in scalers and 'mean' in scalers[feat]:
            count = scalers[feat]['count']
            if count > 0:
                print(f"  ✅ {feat}: {count:,} samples")
            else:
                print(f"  ⚠️ {feat}: default values (no samples)")
                missing_features.append(feat)
        else:
            print(f"  ❌ {feat}: not found in scalers")
            missing_features.append(feat)

    # 8. 检查数据质量
    print(f"\n🔍 Data Quality Check:")

    # 检查是否有足够的样本
    total_samples = sum(scalers[feat]['count'] for feat in scalers
                       if 'count' in scalers[feat] and scalers[feat]['count'] > 0)

    if total_samples > 0:
        print(f"  ✅ Total valid samples: {total_samples:,}")

        # 检查标准差是否合理
        extreme_std_features = []
        for feat in expected_continuous:
            if feat in scalers and 'std' in scalers[feat]:
                std = scalers[feat]['std']
                if std < 0.001 or std > 1000:
                    extreme_std_features.append((feat, std))

        if extreme_std_features:
            print(f"  ⚠️ Features with extreme std values:")
            for feat, std in extreme_std_features:
                print(f"    {feat}: std={std:.6f}")
        else:
            print(f"  ✅ All std values are reasonable")

    else:
        print(f"  ❌ No valid samples found!")
        return False

    # 9. 测试scaler应用
    print(f"\n🧪 Testing Scaler Application:")

    try:
        # 创建测试数据
        test_X = np.random.randn(len(feature_order), 10, 10).astype(np.float32)

        # 应用scaler
        from Generate_NO2_standardization_parameter import apply_no2_scaler
        X_scaled = apply_no2_scaler(test_X, feature_order, scaler_file)

        print(f"  ✅ Scaler application test passed")
        print(f"  Input shape: {test_X.shape}")
        print(f"  Output shape: {X_scaled.shape}")

        # 检查标准化效果
        for i, feat_name in enumerate(feature_order):
            if feat_name in scalers and 'mean' in scalers[feat_name]:
                mean_scaled = np.mean(X_scaled[i])
                std_scaled = np.std(X_scaled[i])
                print(f"  {feat_name}: scaled_mean={mean_scaled:.4f}, scaled_std={std_scaled:.4f}")
                break  # 只显示第一个连续特征

    except Exception as e:
        print(f"  ❌ Scaler application test failed: {e}")
        return False

    # 10. 最终评估
    print(f"\n🎯 Final Assessment:")

    success_criteria = [
        (continuous_count == 14, f"14 continuous features processed"),
        (categorical_count == 10, f"10 categorical features identified"),
        (non_standardized_count == 5, f"5 non-standardized features identified"),
        (total_samples > 100000, f"Sufficient samples ({total_samples:,})"),
        (len(missing_features) == 0, f"No missing key features"),
        (len(extreme_std_features) == 0, f"No extreme std values")
    ]

    passed = 0
    for criterion, description in success_criteria:
        if criterion:
            print(f"  ✅ {description}")
            passed += 1
        else:
            print(f"  ❌ {description}")

    success_rate = passed / len(success_criteria) * 100
    print(f"\n📊 Overall Success Rate: {success_rate:.1f}% ({passed}/{len(success_criteria)})")

    if success_rate >= 80:
        print(f"🎉 NO2 Scaler validation PASSED!")
        return True
    else:
        print(f"⚠️ NO2 Scaler validation needs attention")
        return False

def visualize_scaler_statistics():
    """可视化scaler统计信息"""
    print(f"\n📊 Visualizing Scaler Statistics...")

    scaler_file = "/content/drive/MyDrive/Scalers/no2_scalers_2019_2021.npz"

    if not os.path.exists(scaler_file):
        print(f"❌ Scaler file not found for visualization")
        return

    try:
        with np.load(scaler_file, allow_pickle=True) as data:
            scalers = data['scalers'].item()
            feature_order = data['feature_order']

        # 提取连续特征的统计信息
        continuous_features = []
        means = []
        stds = []
        counts = []

        for feat_name in feature_order:
            if feat_name in scalers and 'mean' in scalers[feat_name]:
                continuous_features.append(feat_name)
                means.append(scalers[feat_name]['mean'])
                stds.append(scalers[feat_name]['std'])
                counts.append(scalers[feat_name]['count'])

        if not continuous_features:
            print(f"❌ No continuous features found for visualization")
            return

        # 创建图表
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

        # 1. 均值分布
        ax1.bar(range(len(continuous_features)), means)
        ax1.set_title('Feature Means')
        ax1.set_xlabel('Features')
        ax1.set_ylabel('Mean Value')
        ax1.set_xticks(range(len(continuous_features)))
        ax1.set_xticklabels(continuous_features, rotation=45, ha='right')

        # 2. 标准差分布
        ax2.bar(range(len(continuous_features)), stds)
        ax2.set_title('Feature Standard Deviations')
        ax2.set_xlabel('Features')
        ax2.set_ylabel('Standard Deviation')
        ax2.set_xticks(range(len(continuous_features)))
        ax2.set_xticklabels(continuous_features, rotation=45, ha='right')

        # 3. 样本数量分布
        ax3.bar(range(len(continuous_features)), counts)
        ax3.set_title('Sample Counts')
        ax3.set_xlabel('Features')
        ax3.set_ylabel('Sample Count')
        ax3.set_xticks(range(len(continuous_features)))
        ax3.set_xticklabels(continuous_features, rotation=45, ha='right')
        ax3.set_yscale('log')

        # 4. 均值vs标准差散点图
        ax4.scatter(means, stds, s=100, alpha=0.7)
        ax4.set_title('Mean vs Standard Deviation')
        ax4.set_xlabel('Mean')
        ax4.set_ylabel('Standard Deviation')

        # 添加特征标签
        for i, feat in enumerate(continuous_features):
            ax4.annotate(feat, (means[i], stds[i]), xytext=(5, 5),
                        textcoords='offset points', fontsize=8)

        plt.tight_layout()
        plt.savefig('/content/drive/MyDrive/Scalers/no2_scaler_statistics.png',
                   dpi=300, bbox_inches='tight')
        plt.show()

        print(f"✅ Visualization saved to: /content/drive/MyDrive/Scalers/no2_scaler_statistics.png")

    except Exception as e:
        print(f"❌ Visualization failed: {e}")

if __name__ == "__main__":
    # 运行验证
    success = validate_no2_scaler_results()

    if success:
        # 如果验证成功，运行可视化
        visualize_scaler_statistics()

    print(f"\n🎯 Next Steps:")
    if success:
        print(f"  1. ✅ NO2 scaler is ready for training")
        print(f"  2. 🚀 Run SO2 scaler: python 'Generate SO2 standardization parameter.py'")
        print(f"  3. 🎯 Proceed with model training")
    else:
        print(f"  1. 🔧 Fix issues identified in validation")
        print(f"  2. 🔄 Re-run NO2 scaler if needed")
        print(f"  3. ✅ Validate again before proceeding")


🔍 NO2 Scaler Results Validation
✅ Scaler file found: /content/drive/MyDrive/Scalers/no2_scalers_2019_2021.npz
✅ Successfully loaded scaler data

📊 Metadata:
  Data type: no2
  Years: [2019, 2020, 2021]
  Mask logic: mask==1 is valid, mask==0 is invalid (CONFIRMED FROM SOURCE CODE)
  Total features: 29
  Continuous features: 14

📋 Feature Order (29 features):
   1. dem
   2. slope
   3. pop
   4. lulc_class_0
   5. lulc_class_1
   6. lulc_class_2
   7. lulc_class_3
   8. lulc_class_4
   9. lulc_class_5
  10. lulc_class_6
  11. lulc_class_7
  12. lulc_class_8
  13. lulc_class_9
  14. sin_doy
  15. cos_doy
  16. weekday_weight
  17. u10
  18. v10
  19. blh
  20. tp
  21. t2m
  22. sp
  23. str
  24. ssr_clr
  25. ws
  26. wd_sin
  27. wd_cos
  28. no2_lag_1day
  29. no2_neighbor

🔢 Scaler Parameters:
  ✅ dem: mean=606.2273, std=771.7322, count=1,052,695
  ✅ slope: mean=11.3112, std=11.0288, count=994,379
  ✅ pop: mean=135.9125, std=369.5149, count=992,667
  📊 lulc_class_0: categorical (on

In [ ]:
# Diagnose NO2 Feature Issues
import os
import numpy as np
from collections import defaultdict

def diagnose_no2_feature_issues():
    """诊断NO2特征问题"""
    print("🔍 NO2 Feature Issues Diagnosis")
    print("=" * 50)

    BASE_NO2 = "/content/drive/MyDrive/Feature_Stacks"

    # 检查的特征
    problem_features = ['sp', 'ssr_clr', 'str']

    # 统计信息
    feature_stats = defaultdict(lambda: {
        'total_files': 0,
        'files_with_feature': 0,
        'files_with_nan': 0,
        'files_with_finite': 0,
        'total_pixels': 0,
        'finite_pixels': 0,
        'nan_pixels': 0,
        'zero_pixels': 0,
        'negative_pixels': 0,
        'positive_pixels': 0
    })

    # 检查几个样本文件
    sample_files = []
    for year in [2019, 2020, 2021]:
        year_dir = os.path.join(BASE_NO2, f"NO2_{year}")
        if os.path.exists(year_dir):
            files = [f for f in os.listdir(year_dir) if f.endswith('.npz')]
            if files:
                sample_files.append(os.path.join(year_dir, files[0]))  # 取第一个文件
                if len(sample_files) >= 3:  # 最多检查3个文件
                    break

    print(f"📁 Checking {len(sample_files)} sample files:")
    for file_path in sample_files:
        print(f"  {os.path.basename(file_path)}")

    print(f"\n🔍 Detailed Analysis:")

    for file_path in sample_files:
        print(f"\n📄 File: {os.path.basename(file_path)}")

        try:
            with np.load(file_path) as data:
                print(f"  📊 Available keys: {list(data.files)}")

                # 检查目标数据和掩膜
                if 'no2_target' in data.files and 'no2_mask' in data.files:
                    target = data['no2_target']
                    mask = data['no2_mask']

                    # 计算有效像素
                    valid_mask = (mask == 1) & (target > 0)
                    valid_pixels = valid_mask.sum()
                    total_pixels = mask.size

                    print(f"  🎯 Valid pixels: {valid_pixels:,}/{total_pixels:,} ({100*valid_pixels/total_pixels:.1f}%)")

                    # 检查问题特征
                    for feat_name in problem_features:
                        if feat_name in data.files:
                            feature_data = data[feat_name]

                            # 在有效像素上的统计
                            valid_feature_data = feature_data[valid_mask]

                            # 统计信息
                            total_valid = len(valid_feature_data)
                            finite_count = np.isfinite(valid_feature_data).sum()
                            nan_count = np.isnan(valid_feature_data).sum()
                            zero_count = (valid_feature_data == 0).sum()
                            negative_count = (valid_feature_data < 0).sum()
                            positive_count = (valid_feature_data > 0).sum()

                            print(f"    {feat_name}:")
                            print(f"      Total valid pixels: {total_valid:,}")
                            print(f"      Finite values: {finite_count:,} ({100*finite_count/total_valid:.1f}%)")
                            print(f"      NaN values: {nan_count:,} ({100*nan_count/total_valid:.1f}%)")
                            print(f"      Zero values: {zero_count:,} ({100*zero_count/total_valid:.1f}%)")
                            print(f"      Negative values: {negative_count:,} ({100*negative_count/total_valid:.1f}%)")
                            print(f"      Positive values: {positive_count:,} ({100*positive_count/total_valid:.1f}%)")

                            if finite_count > 0:
                                finite_data = valid_feature_data[np.isfinite(valid_feature_data)]
                                print(f"      Range: [{finite_data.min():.6f}, {finite_data.max():.6f}]")
                                print(f"      Mean: {finite_data.mean():.6f}")
                                print(f"      Std: {finite_data.std():.6f}")

                            # 更新统计
                            feature_stats[feat_name]['total_files'] += 1
                            feature_stats[feat_name]['files_with_feature'] += 1
                            feature_stats[feat_name]['total_pixels'] += total_valid
                            feature_stats[feat_name]['finite_pixels'] += finite_count
                            feature_stats[feat_name]['nan_pixels'] += nan_count

                            if finite_count > 0:
                                feature_stats[feat_name]['files_with_finite'] += 1
                            if nan_count > 0:
                                feature_stats[feat_name]['files_with_nan'] += 1
                        else:
                            print(f"    {feat_name}: ❌ NOT FOUND in file")
                            feature_stats[feat_name]['total_files'] += 1

        except Exception as e:
            print(f"  ❌ Error loading file: {e}")

    # 总结统计
    print(f"\n📊 Summary Statistics:")
    for feat_name in problem_features:
        stats = feature_stats[feat_name]
        print(f"\n  {feat_name}:")
        print(f"    Files checked: {stats['total_files']}")
        print(f"    Files with feature: {stats['files_with_feature']}")
        print(f"    Files with finite values: {stats['files_with_finite']}")
        print(f"    Files with NaN values: {stats['files_with_nan']}")

        if stats['total_pixels'] > 0:
            finite_ratio = stats['finite_pixels'] / stats['total_pixels']
            nan_ratio = stats['nan_pixels'] / stats['total_pixels']
            print(f"    Finite ratio: {finite_ratio:.1%}")
            print(f"    NaN ratio: {nan_ratio:.1%}")

            if finite_ratio < 0.1:
                print(f"    ⚠️ Very low finite ratio - consider removing from CONTINUOUS_FEATURES")
            elif nan_ratio > 0.9:
                print(f"    ⚠️ Very high NaN ratio - consider removing from CONTINUOUS_FEATURES")
            else:
                print(f"    ✅ Data quality acceptable")

def check_feature_naming_consistency():
    """检查特征命名一致性"""
    print(f"\n🔍 Feature Naming Consistency Check:")
    print("=" * 50)

    BASE_NO2 = "/content/drive/MyDrive/Feature_Stacks"

    # 检查不同年份的文件键名
    all_keys = set()
    year_keys = {}

    for year in [2019, 2020, 2021]:
        year_dir = os.path.join(BASE_NO2, f"NO2_{year}")
        if os.path.exists(year_dir):
            files = [f for f in os.listdir(year_dir) if f.endswith('.npz')]
            if files:
                # 检查第一个文件
                file_path = os.path.join(year_dir, files[0])
                try:
                    with np.load(file_path) as data:
                        keys = set(data.files)
                        all_keys.update(keys)
                        year_keys[year] = keys
                        print(f"  {year}: {len(keys)} keys")
                except Exception as e:
                    print(f"  {year}: Error - {e}")

    # 检查关键特征
    key_features = ['sp', 'ssr_clr', 'ssr_clear', 'str']
    print(f"\n🎯 Key Feature Analysis:")

    for feat in key_features:
        found_in_years = []
        for year, keys in year_keys.items():
            if feat in keys:
                found_in_years.append(year)

        if found_in_years:
            print(f"  {feat}: Found in years {found_in_years}")
        else:
            print(f"  {feat}: ❌ NOT FOUND in any year")

    # 检查可能的命名变体
    print(f"\n🔍 Possible naming variants:")
    for year, keys in year_keys.items():
        radiation_keys = [k for k in keys if 'ssr' in k.lower() or 'radiation' in k.lower()]
        pressure_keys = [k for k in keys if 'sp' in k.lower() or 'pressure' in k.lower()]
        str_keys = [k for k in keys if 'str' in k.lower()]

        if radiation_keys:
            print(f"  {year} radiation-related keys: {radiation_keys}")
        if pressure_keys:
            print(f"  {year} pressure-related keys: {pressure_keys}")
        if str_keys:
            print(f"  {year} str-related keys: {str_keys}")

if __name__ == "__main__":
    diagnose_no2_feature_issues()
    check_feature_naming_consistency()

    print(f"\n💡 Recommendations:")
    print("=" * 50)
    print("1. If sp/ssr_clr/str have >90% NaN values → Remove from CONTINUOUS_FEATURES")
    print("2. If naming inconsistency found → Update feature names in scaler")
    print("3. If data coverage is poor → Consider alternative features or imputation")
    print("4. For str with low samples → Consider removing or using alternative radiation variable")


🔍 NO2 Feature Issues Diagnosis
📁 Checking 3 sample files:
  NO2_stack_20190102.npz
  NO2_stack_20200101.npz
  NO2_stack_20210101.npz

🔍 Detailed Analysis:

📄 File: NO2_stack_20190102.npz
  📊 Available keys: ['no2_target', 'no2_mask', 'dem', 'slope', 'pop', 'lulc_class_0', 'lulc_class_1', 'lulc_class_2', 'lulc_class_3', 'lulc_class_4', 'lulc_class_5', 'lulc_class_6', 'lulc_class_7', 'lulc_class_8', 'lulc_class_9', 'sin_doy', 'cos_doy', 'weekday_weight', 'u10', 'v10', 'blh', 'tp', 't2m', 'sp', 'str', 'ssr_clr', 'ws', 'wd_sin', 'wd_cos', 'no2_lag_1day', 'no2_neighbor', 'coverage_rate', 'valid_pixels', 'total_pixels', 'coverage_in_aoi', 'aoi_pixels', 'trainable']
  🎯 Valid pixels: 73,631/186,300 (39.5%)
    sp:
      Total valid pixels: 73,631
      Finite values: 0 (0.0%)
      NaN values: 0 (0.0%)
      Zero values: 0 (0.0%)
      Negative values: 0 (0.0%)
      Positive values: 73,631 (100.0%)
    ssr_clr:
      Total valid pixels: 73,631
      Finite values: 0 (0.0%)
      NaN values: 

In [ ]:
# Quick Check Problem Features
import os
import numpy as np

def quick_check_problem_features():
    """快速检查问题特征"""
    print("🔍 Quick Check Problem Features")
    print("=" * 40)

    BASE_NO2 = "/content/drive/MyDrive/Feature_Stacks"

    # 检查一个样本文件
    sample_file = None
    for year in [2019, 2020, 2021]:
        year_dir = os.path.join(BASE_NO2, f"NO2_{year}")
        if os.path.exists(year_dir):
            files = [f for f in os.listdir(year_dir) if f.endswith('.npz')]
            if files:
                sample_file = os.path.join(year_dir, files[0])
                break

    if not sample_file:
        print("❌ No sample file found")
        return

    print(f"📄 Checking file: {os.path.basename(sample_file)}")

    try:
        with np.load(sample_file) as data:
            print(f"📊 Available keys: {list(data.files)}")

            # 检查目标数据和掩膜
            if 'no2_target' in data.files and 'no2_mask' in data.files:
                target = data['no2_target']
                mask = data['no2_mask']

                # 计算有效像素
                valid_mask = (mask == 1) & (target > 0)
                valid_pixels = valid_mask.sum()
                total_pixels = mask.size

                print(f"🎯 Valid pixels: {valid_pixels:,}/{total_pixels:,} ({100*valid_pixels/total_pixels:.1f}%)")

                # 检查问题特征
                problem_features = ['sp', 'ssr_clr', 'str']

                for feat_name in problem_features:
                    print(f"\n🔍 {feat_name}:")

                    if feat_name in data.files:
                        feature_data = data[feat_name]
                        valid_feature_data = feature_data[valid_mask]

                        # 基本统计
                        total_valid = len(valid_feature_data)
                        finite_count = np.isfinite(valid_feature_data).sum()
                        nan_count = np.isnan(valid_feature_data).sum()

                        print(f"  Total valid pixels: {total_valid:,}")
                        print(f"  Finite values: {finite_count:,} ({100*finite_count/total_valid:.1f}%)")
                        print(f"  NaN values: {nan_count:,} ({100*nan_count/total_valid:.1f}%)")

                        if finite_count > 0:
                            finite_data = valid_feature_data[np.isfinite(valid_feature_data)]
                            print(f"  Range: [{finite_data.min():.6f}, {finite_data.max():.6f}]")
                            print(f"  Mean: {finite_data.mean():.6f}")
                            print(f"  Std: {finite_data.std():.6f}")

                            # 判断数据质量
                            if finite_count / total_valid < 0.1:
                                print(f"  ⚠️ Very low finite ratio - should be removed from CONTINUOUS_FEATURES")
                            elif nan_count / total_valid > 0.9:
                                print(f"  ⚠️ Very high NaN ratio - should be removed from CONTINUOUS_FEATURES")
                            else:
                                print(f"  ✅ Data quality acceptable")
                        else:
                            print(f"  ❌ No finite values - should be removed from CONTINUOUS_FEATURES")
                    else:
                        print(f"  ❌ Feature not found in file")

                        # 检查可能的命名变体
                        possible_names = []
                        if feat_name == 'ssr_clr':
                            possible_names = ['ssr_clear', 'ssr', 'clear_sky_radiation']
                        elif feat_name == 'sp':
                            possible_names = ['pressure', 'surface_pressure', 'sp_pressure']
                        elif feat_name == 'str':
                            possible_names = ['surface_thermal_radiation', 'thermal_radiation', 'str_radiation']

                        for possible_name in possible_names:
                            if possible_name in data.files:
                                print(f"  💡 Found possible variant: {possible_name}")

    except Exception as e:
        print(f"❌ Error loading file: {e}")

if __name__ == "__main__":
    quick_check_problem_features()


🔍 Quick Check Problem Features
📄 Checking file: NO2_stack_20190102.npz
📊 Available keys: ['no2_target', 'no2_mask', 'dem', 'slope', 'pop', 'lulc_class_0', 'lulc_class_1', 'lulc_class_2', 'lulc_class_3', 'lulc_class_4', 'lulc_class_5', 'lulc_class_6', 'lulc_class_7', 'lulc_class_8', 'lulc_class_9', 'sin_doy', 'cos_doy', 'weekday_weight', 'u10', 'v10', 'blh', 'tp', 't2m', 'sp', 'str', 'ssr_clr', 'ws', 'wd_sin', 'wd_cos', 'no2_lag_1day', 'no2_neighbor', 'coverage_rate', 'valid_pixels', 'total_pixels', 'coverage_in_aoi', 'aoi_pixels', 'trainable']
🎯 Valid pixels: 73,631/186,300 (39.5%)

🔍 sp:
  Total valid pixels: 73,631
  Finite values: 0 (0.0%)
  NaN values: 0 (0.0%)
  ❌ No finite values - should be removed from CONTINUOUS_FEATURES

🔍 ssr_clr:
  Total valid pixels: 73,631
  Finite values: 0 (0.0%)
  NaN values: 0 (0.0%)
  ❌ No finite values - should be removed from CONTINUOUS_FEATURES

🔍 str:
  Total valid pixels: 73,631
  Finite values: 0 (0.0%)
  NaN values: 0 (0.0%)
  ❌ No finite valu

In [ ]:
%%writefile /content/drive/MyDrive/GeoGapFiller/regenerate_no2_scaler_fixed.py
#!/usr/bin/env python3
"""
NO2 Scaler重新生成脚本（修复版）
Regenerate NO2 Scaler (Fixed Version)

使用修复后的特征栈重新生成NO2标准化参数
"""

import os
import numpy as np
import sys
from datetime import datetime
import time

# 添加当前目录到Python路径
sys.path.append(os.path.dirname(os.path.abspath(__file__)))

def regenerate_no2_scaler():
    """重新生成NO2 scaler"""
    print("🚀 Regenerating NO2 Scaler (Fixed Version)")
    print("=" * 60)

    # 配置路径
    base_path = "/content/drive/MyDrive"
    feature_stack_dir = os.path.join(base_path, "Feature_Stacks", "NO2_Independent")
    output_dir = os.path.join(base_path, "Scalers")

    # 确保输出目录存在
    os.makedirs(output_dir, exist_ok=True)

    # 输出文件路径
    scaler_file = os.path.join(output_dir, "no2_scalers_2019_2021_fixed.npz")

    print(f"📁 Feature stack directory: {feature_stack_dir}")
    print(f"📁 Output directory: {output_dir}")
    print(f"📁 Scaler file: {scaler_file}")

    # 检查特征栈目录
    if not os.path.exists(feature_stack_dir):
        print(f"❌ Feature stack directory not found: {feature_stack_dir}")
        return False

    # 获取特征栈文件列表
    feature_files = [f for f in os.listdir(feature_stack_dir) if f.startswith("NO2_stack_") and f.endswith(".npz")]
    feature_files.sort()

    print(f"📊 Found {len(feature_files)} feature stack files")

    if len(feature_files) == 0:
        print("❌ No feature stack files found!")
        return False

    # 定义特征类型
    CONTINUOUS_FEATURES = [
        'dem', 'slope', 'pop', 'u10', 'v10', 'blh', 'tp', 't2m',
        'sp', 'str', 'ssr_clr', 'ws', 'no2_lag_1day', 'no2_neighbor'
    ]

    CATEGORICAL_FEATURES = [
        'lulc_class_0', 'lulc_class_1', 'lulc_class_2', 'lulc_class_3', 'lulc_class_4',
        'lulc_class_5', 'lulc_class_6', 'lulc_class_7', 'lulc_class_8', 'lulc_class_9'
    ]

    NON_STANDARDIZED = [
        'wd_sin', 'wd_cos', 'sin_doy', 'cos_doy', 'weekday_weight'
    ]

    # 初始化统计信息
    scalers = {}
    feature_order = []
    total_samples = 0
    processed_files = 0
    failed_files = 0

    # 为每个连续特征初始化统计信息
    for feat in CONTINUOUS_FEATURES:
        scalers[feat] = {
            'sum': 0.0,
            'sum_sq': 0.0,
            'count': 0,
            'min': float('inf'),
            'max': float('-inf')
        }

    print(f"\n🔍 Processing feature stack files...")
    start_time = time.time()

    # 处理每个特征栈文件
    for i, filename in enumerate(feature_files):
        try:
            file_path = os.path.join(feature_stack_dir, filename)

            # 加载特征栈
            with np.load(file_path, allow_pickle=True) as data:
                # 检查必要的键
                if 'no2_target' not in data or 'no2_mask' not in data:
                    print(f"   ⚠️ {filename}: Missing target or mask, skipping")
                    failed_files += 1
                    continue

                # 获取目标变量和掩膜
                y = data['no2_target']
                mask = data['no2_mask']

                # 创建有效掩膜（mask==1且y>0）
                valid_mask = (mask == 1) & (y > 0)

                if not np.any(valid_mask):
                    print(f"   ⚠️ {filename}: No valid pixels, skipping")
                    failed_files += 1
                    continue

                # 构建特征矩阵
                X = []
                current_feature_order = []

                # 添加连续特征
                for feat in CONTINUOUS_FEATURES:
                    if feat in data:
                        X.append(data[feat])
                        current_feature_order.append(feat)
                    else:
                        print(f"   ⚠️ {filename}: Missing continuous feature {feat}")
                        # 使用NaN填充
                        X.append(np.full_like(y, np.nan))
                        current_feature_order.append(feat)

                # 添加分类特征（one-hot编码）
                for feat in CATEGORICAL_FEATURES:
                    if feat in data:
                        X.append(data[feat])
                        current_feature_order.append(feat)
                    else:
                        print(f"   ⚠️ {filename}: Missing categorical feature {feat}")
                        # 使用0填充
                        X.append(np.zeros_like(y))
                        current_feature_order.append(feat)

                # 添加非标准化特征
                for feat in NON_STANDARDIZED:
                    if feat in data:
                        X.append(data[feat])
                        current_feature_order.append(feat)
                    else:
                        print(f"   ⚠️ {filename}: Missing non-standardized feature {feat}")
                        # 使用默认值填充
                        if feat in ['wd_sin', 'wd_cos']:
                            X.append(np.zeros_like(y))
                        elif feat in ['sin_doy', 'cos_doy']:
                            X.append(np.zeros_like(y))
                        elif feat == 'weekday_weight':
                            X.append(np.ones_like(y))
                        current_feature_order.append(feat)

                # 转换为numpy数组
                X = np.array(X)

                # 确保特征顺序一致
                if not feature_order:
                    feature_order = current_feature_order
                elif feature_order != current_feature_order:
                    print(f"   ⚠️ {filename}: Feature order mismatch, skipping")
                    failed_files += 1
                    continue

                # 展平数据
                X_flat = X.reshape(X.shape[0], -1)
                valid_mask_flat = valid_mask.flatten()

                # 更新统计信息
                for j, feat in enumerate(CONTINUOUS_FEATURES):
                    if j < X_flat.shape[0]:
                        feat_data = X_flat[j, valid_mask_flat]
                        finite_mask = np.isfinite(feat_data)

                        if np.any(finite_mask):
                            finite_data = feat_data[finite_mask]
                            scalers[feat]['sum'] += np.sum(finite_data)
                            scalers[feat]['sum_sq'] += np.sum(finite_data**2)
                            scalers[feat]['count'] += len(finite_data)
                            scalers[feat]['min'] = min(scalers[feat]['min'], np.min(finite_data))
                            scalers[feat]['max'] = max(scalers[feat]['max'], np.max(finite_data))

                processed_files += 1
                total_samples += np.sum(valid_mask)

                if (i + 1) % 100 == 0:
                    elapsed = time.time() - start_time
                    rate = (i + 1) / elapsed if elapsed > 0 else 0
                    print(f"   📊 Processed {i + 1}/{len(feature_files)} files, rate: {rate:.1f} files/sec")

        except Exception as e:
            print(f"   ❌ {filename}: Error - {e}")
            failed_files += 1

    # 计算最终统计信息
    print(f"\n🔢 Computing final statistics...")

    final_scalers = {}
    for feat in CONTINUOUS_FEATURES:
        if scalers[feat]['count'] > 0:
            mean = scalers[feat]['sum'] / scalers[feat]['count']
            variance = (scalers[feat]['sum_sq'] / scalers[feat]['count']) - (mean ** 2)
            std = np.sqrt(max(0, variance))

            final_scalers[feat] = {
                'mean': mean,
                'std': std,
                'count': scalers[feat]['count'],
                'min': scalers[feat]['min'],
                'max': scalers[feat]['max']
            }
        else:
            # 没有样本的特征使用默认值
            final_scalers[feat] = {
                'mean': 0.0,
                'std': 1.0,
                'count': 0,
                'min': 0.0,
                'max': 0.0
            }

    # 保存scaler
    metadata = {
        'years': [2019, 2020, 2021],
        'total_files': len(feature_files),
        'processed_files': processed_files,
        'failed_files': failed_files,
        'total_samples': total_samples,
        'processing_time': time.time() - start_time,
        'mask_logic': 'mask==1 is valid, mask==0 is invalid',
        'target_logic': 'y > 0 for physical validity'
    }

    np.savez_compressed(
        scaler_file,
        scalers=final_scalers,
        feature_order=feature_order,
        metadata=metadata
    )

    # 显示结果
    print(f"\n📊 NO2 Scaler Regeneration Summary:")
    print(f"   - Total files: {len(feature_files)}")
    print(f"   - Processed: {processed_files}")
    print(f"   - Failed: {failed_files}")
    print(f"   - Success rate: {processed_files/len(feature_files)*100:.1f}%")
    print(f"   - Total samples: {total_samples:,}")
    print(f"   - Processing time: {(time.time() - start_time)/60:.1f} minutes")
    print(f"   - Scaler file: {scaler_file}")

    # 显示关键特征统计
    print(f"\n🔍 Key Features Statistics:")
    key_features = ['sp', 'str', 'ssr_clr', 'dem', 't2m', 'u10', 'v10']
    for feat in key_features:
        if feat in final_scalers:
            stats = final_scalers[feat]
            print(f"   {feat}: mean={stats['mean']:.4f}, std={stats['std']:.4f}, count={stats['count']:,}")

    return True

def main():
    """主函数"""
    print("🚀 NO2 Scaler Regeneration (Fixed Version)")
    print("=" * 60)

    # 确认开始
    response = input("❓ Do you want to regenerate NO2 scaler? (y/n): ").lower().strip()

    if response == 'y':
        success = regenerate_no2_scaler()

        if success:
            print("\n🎉 NO2 Scaler regeneration completed successfully!")
            print("📋 Next steps:")
            print("   1. ✅ Scaler regenerated with fixed data")
            print("   2. 🔄 Ready for model training")
            print("   3. 🔄 Ready for performance evaluation")
        else:
            print("\n❌ NO2 Scaler regeneration failed!")
    else:
        print("\n⏸️ NO2 Scaler regeneration cancelled")

if __name__ == "__main__":
    main()


Writing /content/drive/MyDrive/GeoGapFiller/regenerate_no2_scaler_fixed.py


In [ ]:
!python /content/drive/MyDrive/GeoGapFiller/regenerate_no2_scaler_fixed.py

In [6]:
#!/usr/bin/env python3
"""
NO2 Scaler for Dictionary Structure
专门处理NO2字典式.npz文件结构的scaler生成

NO2文件结构: {'no2_target': array, 'dem': array, 'u10': array, ...}
需要先构建特征矩阵，再进行标准化
"""

import os
import numpy as np
import glob
from datetime import datetime
import time
from collections import defaultdict

def build_feature_matrix_from_dict(data_dict, feature_order):
    """
    从字典结构构建特征矩阵

    Args:
        data_dict: NO2字典式数据 {'no2_target': array, 'dem': array, ...}
        feature_order: 特征顺序列表

    Returns:
        X: 特征矩阵 (C, H, W)
    """
    X_list = []

    for feat_name in feature_order:
        if feat_name in data_dict:
            X_list.append(data_dict[feat_name].astype(np.float32))
        else:
            # 处理缺失特征
            if feat_name.startswith('lulc_class_'):
                # LULC特征缺失用0填充
                fill_value = 0.0
            elif feat_name in ['sin_doy', 'cos_doy', 'wd_sin', 'wd_cos']:
                # 时间/风向特征缺失用0填充
                fill_value = 0.0
            elif feat_name == 'weekday_weight':
                # 工作日权重缺失用1填充
                fill_value = 1.0
            else:
                # 其他特征缺失用NaN填充
                fill_value = np.nan

            # 获取维度 - 改进：优先从no2_target推断
            if X_list:
                H, W = X_list[0].shape
            elif 'no2_target' in data_dict:
                H, W = data_dict['no2_target'].shape
            elif 'no2_mask' in data_dict:
                H, W = data_dict['no2_mask'].shape
            else:
                H, W = 300, 621  # 最后兜底

            X_list.append(np.full((H, W), fill_value, dtype=np.float32))

    return np.stack(X_list, axis=0)  # (C, H, W)

def compute_no2_scaler_dictionary_structure(years=[2019, 2020, 2021], sample_per_file=1000):
    """
    计算NO2标准化参数 - 专门处理字典结构

    Args:
        years: 训练年份
        sample_per_file: 每个文件采样的像素数
    """
    print("🚀 Computing NO2 Scaler for Dictionary Structure")
    print("=" * 60)
    print("📋 NO2 file structure: Dictionary format")
    print("📋 Process: Build feature matrix → Compute statistics → Generate scaler")

    # 设置随机种子确保可复现性
    np.random.seed(42)

    # 配置路径
    base_path = "/content/drive/MyDrive"
    feature_stack_base = os.path.join(base_path, "Feature_Stacks")
    output_dir = os.path.join(base_path, "Scalers")
    os.makedirs(output_dir, exist_ok=True)

    # 定义特征类型和顺序
    CONTINUOUS_FEATURES = [
        # 气象特征
        'u10', 'v10', 'blh', 'tp', 't2m', 'sp', 'str', 'ssr_clr',
        # 风场派生特征
        'ws',  # 移除 wd_sin, wd_cos
        # NO2相关特征
        'no2_lag_1day', 'no2_neighbor',
        # 静态特征
        'dem', 'slope', 'pop'
    ]

    CATEGORICAL_FEATURES = [
        'lulc_class_0', 'lulc_class_1', 'lulc_class_2', 'lulc_class_3', 'lulc_class_4',
        'lulc_class_5', 'lulc_class_6', 'lulc_class_7', 'lulc_class_8', 'lulc_class_9'
    ]

    NON_STANDARDIZED = [
        'sin_doy', 'cos_doy', 'weekday_weight',
        'wd_sin', 'wd_cos'  # 新增：风向特征不标准化
    ]

    # 完整的特征顺序
    feature_order = (
        CONTINUOUS_FEATURES +
        CATEGORICAL_FEATURES +
        NON_STANDARDIZED
    )

    print(f"📊 Feature configuration:")
    print(f"   - Continuous features: {len(CONTINUOUS_FEATURES)}")
    print(f"   - Categorical features: {len(CATEGORICAL_FEATURES)}")
    print(f"   - Non-standardized features: {len(NON_STANDARDIZED)}")
    print(f"   - Total features: {len(feature_order)}")
    print(f"   - 改进: 风向特征(wd_sin, wd_cos)不标准化，保持[-1,1]范围")
    print(f"   - 改进: 有效像素判定可配置，默认不过滤y<=0")

    # 初始化统计信息
    stats = defaultdict(lambda: {'sum': 0.0, 'sumsq': 0.0, 'count': 0})

    total_files = 0
    processed_files = 0
    failed_files = 0

    print(f"\n🔍 Processing NO2 dictionary files...")
    start_time = time.time()

    # 按年份处理
    for year in years:
        year_dir = os.path.join(feature_stack_base, f"NO2_{year}")
        if not os.path.exists(year_dir):
            print(f"⚠️ Year directory not found: {year_dir}")
            continue

        files = [f for f in os.listdir(year_dir) if f.startswith("NO2_stack_") and f.endswith(".npz")]
        files.sort()
        total_files += len(files)

        print(f"📅 Processing {year}: {len(files)} files")

        for i, filename in enumerate(files):
            try:
                file_path = os.path.join(year_dir, filename)

                # 加载字典式数据
                with np.load(file_path, allow_pickle=True) as data:
                    # 检查必要键
                    if 'no2_target' not in data or 'no2_mask' not in data:
                        print(f"   ⚠️ {filename}: Missing target or mask")
                        failed_files += 1
                        continue

                    # 获取目标变量和掩膜
                    y = data['no2_target']
                    mask = data['no2_mask']

                    # 创建有效掩膜 - 改进版：可配置是否过滤非正值
                    FILTER_NONPOSITIVE_TARGET = False  # 可配置：是否过滤y<=0的像素
                    valid_mask = (mask == 1)
                    if FILTER_NONPOSITIVE_TARGET:
                        valid_mask &= (y > 0)

                    if not np.any(valid_mask):
                        print(f"   ⚠️ {filename}: No valid pixels")
                        failed_files += 1
                        continue

                    # 从字典构建特征矩阵
                    X = build_feature_matrix_from_dict(data, feature_order)

                    # 形状一致性检查
                    if X.shape[0] != len(feature_order):
                        print(f"   ⚠️ {filename}: Feature matrix shape mismatch: X.shape[0]={X.shape[0]}, feature_order length={len(feature_order)}")
                        failed_files += 1
                        continue

                    # 提取有效像素
                    C, H, W = X.shape
                    X_flat = X.reshape(C, -1)  # (C, H*W)
                    valid_flat = valid_mask.ravel()  # (H*W,)
                    valid_pixels = X_flat[:, valid_flat]  # (C, N_valid)

                    if valid_pixels.shape[1] == 0:
                        continue

                    # 随机采样
                    n_samples = min(sample_per_file, valid_pixels.shape[1])
                    if n_samples < valid_pixels.shape[1]:
                        indices = np.random.choice(valid_pixels.shape[1], n_samples, replace=False)
                        sampled = valid_pixels[:, indices]
                    else:
                        sampled = valid_pixels

                    # 过滤NaN/inf
                    finite_mask = np.isfinite(sampled)

                    # 累积统计信息（只对连续特征）
                    for j, feat_name in enumerate(feature_order):
                        if feat_name in CONTINUOUS_FEATURES:
                            values = sampled[j]
                            finite_values = values[finite_mask[j]]

                            if len(finite_values) > 0:
                                # 1-99百分位winsorization
                                q1, q99 = np.percentile(finite_values, [1, 99])
                                clipped = np.clip(finite_values, q1, q99)

                                stats[feat_name]['sum'] += np.sum(clipped)
                                stats[feat_name]['sumsq'] += np.sum(clipped**2)
                                stats[feat_name]['count'] += len(clipped)

                    processed_files += 1

                    if (i + 1) % 50 == 0:
                        elapsed = time.time() - start_time
                        rate = (i + 1) / elapsed if elapsed > 0 else 0
                        progress_pct = (i + 1) / len(files) * 100
                        print(f"   📊 {year}: {i + 1}/{len(files)} files ({progress_pct:.1f}%), rate: {rate:.1f} files/sec")

            except Exception as e:
                print(f"   ❌ {filename}: Error - {e}")
                failed_files += 1

    print(f"\n📊 Processing statistics:")
    print(f"   - Total files: {total_files}")
    print(f"   - Processed: {processed_files}")
    print(f"   - Failed: {failed_files}")
    print(f"   - Success rate: {processed_files/total_files*100:.1f}%")

    # 特征一致性检查
    print(f"\n🔍 Feature consistency check:")
    all_defined_features = set(CONTINUOUS_FEATURES + CATEGORICAL_FEATURES + NON_STANDARDIZED)
    feature_order_set = set(feature_order)

    missing_in_order = all_defined_features - feature_order_set
    extra_in_order = feature_order_set - all_defined_features

    if missing_in_order:
        print(f"   ⚠️ Features defined but missing in feature_order: {missing_in_order}")
    if extra_in_order:
        print(f"   ⚠️ Features in feature_order but not defined: {extra_in_order}")
    if not missing_in_order and not extra_in_order:
        print(f"   ✅ All feature names are consistent!")

    if not stats:
        print("❌ No valid samples found")
        return None, None

    # 计算最终统计信息
    scalers = {}
    print(f"\n🔢 Computing standardization parameters...")

    for feat_name in feature_order:
        if feat_name in CONTINUOUS_FEATURES:
            if feat_name in stats and stats[feat_name]['count'] > 0:
                count = stats[feat_name]['count']
                mean = stats[feat_name]['sum'] / count
                variance = (stats[feat_name]['sumsq'] / count) - (mean**2)
                std = np.sqrt(max(variance, 1e-8))

                scalers[feat_name] = {
                    'mean': float(mean),
                    'std': float(std),
                    'count': int(count)
                }
                print(f"   {feat_name}: mean={mean:.4f}, std={std:.4f}, count={count}")
            else:
                scalers[feat_name] = {'mean': 0.0, 'std': 1.0, 'count': 0}
                print(f"   {feat_name}: No valid samples, using defaults")
        elif feat_name in CATEGORICAL_FEATURES:
            scalers[feat_name] = {'type': 'categorical'}
            print(f"   {feat_name}: categorical (one-hot)")
        elif feat_name in NON_STANDARDIZED:
            scalers[feat_name] = {'type': 'non_standardized'}
            print(f"   {feat_name}: non_standardized")

    return scalers, feature_order

def apply_no2_scaler_dictionary_structure(data_dict, feature_order, scalers):
    """
    对字典式NO2数据应用标准化

    Args:
        data_dict: NO2字典式数据
        feature_order: 特征顺序
        scalers: 标准化参数

    Returns:
        X_scaled: 标准化后的特征矩阵 (C, H, W)
    """
    # 构建特征矩阵
    X = build_feature_matrix_from_dict(data_dict, feature_order)
    X_scaled = X.copy().astype(np.float32)

    # 应用标准化
    for i, feat_name in enumerate(feature_order):
        if feat_name in scalers and 'mean' in scalers[feat_name]:
            # 对连续特征应用z-score标准化
            mean = scalers[feat_name]['mean']
            std = scalers[feat_name]['std']
            X_scaled[i] = (X[i] - mean) / std
        # 跳过分类和非标准化特征

    return X_scaled

def save_no2_scaler_dictionary_structure(scalers, feature_order, years):
    """保存NO2 scaler参数"""
    output_dir = "/content/drive/MyDrive/Scalers"
    os.makedirs(output_dir, exist_ok=True)

    output_file = os.path.join(output_dir, f"no2_scalers_dict_{years[0]}_{years[-1]}.npz")

    metadata = {
        'data_type': 'no2_dictionary',
        'years': years,
        'file_structure': 'dictionary format: {no2_target: array, dem: array, ...}',
        'processing': 'build feature matrix → standardize',
        'mask_logic': 'mask==1 is valid, mask==0 is invalid',
        'target_filtering': 'FILTER_NONPOSITIVE_TARGET=False (includes y=0 as valid)',
        'wind_features': 'wd_sin, wd_cos are non-standardized (preserve [-1,1] range)',
        'total_features': len(feature_order),
        'continuous_features': len([f for f in feature_order if 'mean' in scalers.get(f, {})]),
        'categorical_features': len([f for f in feature_order if scalers.get(f, {}).get('type') == 'categorical']),
        'non_standardized_features': len([f for f in feature_order if scalers.get(f, {}).get('type') == 'non_standardized'])
    }

    np.savez_compressed(
        output_file,
        scalers=scalers,
        feature_order=feature_order,
        metadata=metadata
    )

    print(f"\n✅ NO2 Dictionary Scaler saved to: {output_file}")
    return output_file

def main():
    """主函数"""
    print("🚀 NO2 Scaler for Dictionary Structure")
    print("=" * 60)
    print("📋 This script handles NO2 dictionary-style .npz files")
    print("📋 Process: Dictionary → Feature Matrix → Standardization")

    # 计算scaler
    scalers, feature_order = compute_no2_scaler_dictionary_structure(years=[2019, 2020, 2021])

    if scalers is not None:
        # 保存结果
        output_file = save_no2_scaler_dictionary_structure(scalers, feature_order, [2019, 2020, 2021])

        # 显示统计信息
        print(f"\n📊 NO2 Dictionary Scaler Summary:")
        print(f"   - Total features: {len(feature_order)}")
        print(f"   - Continuous features: {len([f for f in feature_order if 'mean' in scalers.get(f, {})])}")
        print(f"   - Categorical features: {len([f for f in feature_order if scalers.get(f, {}).get('type') == 'categorical'])}")
        print(f"   - Non-standardized features: {len([f for f in feature_order if scalers.get(f, {}).get('type') == 'non_standardized'])}")

        # 显示关键特征参数
        print(f"\n🔍 Key Features Standardization Parameters:")
        key_features = ['dem', 'slope', 'pop', 'u10', 'v10', 't2m', 'no2_lag_1day', 'no2_neighbor']
        for feat in key_features:
            if feat in scalers and 'mean' in scalers[feat]:
                print(f"   {feat}: mean={scalers[feat]['mean']:.4f}, std={scalers[feat]['std']:.4f}")
            elif feat in scalers:
                print(f"   {feat}: {scalers[feat].get('type', 'unknown type')}")

        print(f"\n🎉 NO2 Dictionary Scaler generation completed!")
        print(f"📋 Next steps:")
        print(f"   1. ✅ NO2 scaler ready for dictionary structure")
        print(f"   2. 🔄 Use apply_no2_scaler_dictionary_structure() for standardization")
        print(f"   3. 🔄 Ready for 3D CNN training")

    else:
        print("❌ NO2 scaler computation failed")

if __name__ == "__main__":
    main()


🚀 NO2 Scaler for Dictionary Structure
📋 This script handles NO2 dictionary-style .npz files
📋 Process: Dictionary → Feature Matrix → Standardization
🚀 Computing NO2 Scaler for Dictionary Structure
📋 NO2 file structure: Dictionary format
📋 Process: Build feature matrix → Compute statistics → Generate scaler
📊 Feature configuration:
   - Continuous features: 14
   - Categorical features: 10
   - Non-standardized features: 5
   - Total features: 29
   - 改进: 风向特征(wd_sin, wd_cos)不标准化，保持[-1,1]范围
   - 改进: 有效像素判定可配置，默认不过滤y<=0

🔍 Processing NO2 dictionary files...
📅 Processing 2019: 365 files
   ⚠️ NO2_stack_20190201.npz: No valid pixels
   📊 2019: 50/365 files (13.7%), rate: 8.9 files/sec
   ⚠️ NO2_stack_20190403.npz: No valid pixels
   📊 2019: 100/365 files (27.4%), rate: 9.5 files/sec
   ⚠️ NO2_stack_20190415.npz: No valid pixels
   📊 2019: 150/365 files (41.1%), rate: 9.8 files/sec
   ⚠️ NO2_stack_20190715.npz: No valid pixels
   📊 2019: 200/365 files (54.8%), rate: 9.6 files/sec
   📊 2019:

# 3. Generate SO2 standardization parameters

In [ ]:
# Generate SO2 standardization parameters
import os, numpy as np
from collections import defaultdict

# Create output directory
os.makedirs("/content/drive/MyDrive/Scalers", exist_ok=True)

BASE_SO2 = "/content/drive/MyDrive/Feature_Stacks"
OUT_DIR = "/content/drive/MyDrive/Scalers"

# Continuous features that need standardization (SO2 version)
CONTINUOUS_FEATURES = [
    'u10', 'v10', 'blh', 'tp', 't2m', 'sp', 'str', 'ssr_clear',
    'so2_lag1', 'so2_neighbor', 'so2_climate_prior',  # SO2 related features (removed so2_neighbor_5x5)
    'dem', 'slope', 'population',
    'ws'  # Wind speed
]

# Features that should not be standardized
NON_STANDARDIZED = ['sin_doy', 'cos_doy', 'weekday_weight', 'wd_sin', 'wd_cos']

# Categorical features (one-hot encoded) - SO2 uses lulc_class_10,20,...,100
CATEGORICAL_FEATURES = [
    'lulc_class_10', 'lulc_class_20', 'lulc_class_30', 'lulc_class_40', 'lulc_class_50',
    'lulc_class_60', 'lulc_class_70', 'lulc_class_80', 'lulc_class_90', 'lulc_class_100'
]

def compute_so2_scalers(years=[2019, 2020, 2021], sample_per_file=1000):
    """Compute standardization parameters for SO2"""
    print("🚀 Starting SO2 standardization parameter computation...")
    print(f"Data path: {BASE_SO2}")
    print(f"Training years: {years}")
    print(f"Samples per file: {sample_per_file}")

    # Set random seed for reproducibility
    rng = np.random.default_rng(42)

    # Accumulate statistics
    stats = defaultdict(lambda: {'sum': 0.0, 'sumsq': 0.0, 'count': 0})
    feature_order = None

    total_files = 0
    processed_files = 0

    for year in years:
        year_dir = os.path.join(BASE_SO2, f"SO2_{year}")
        if not os.path.exists(year_dir):
            print(f"⚠️ Year directory not found: {year_dir}")
            continue

        files = [f for f in os.listdir(year_dir) if f.endswith('.npz')]
        total_files += len(files)
        print(f"\n Processing {year}: {len(files)} files")

        for i, fname in enumerate(files):
            if i % 50 == 0:
                print(f"  {i}/{len(files)} ({i/len(files)*100:.1f}%)")

            try:
                with np.load(os.path.join(year_dir, fname)) as data:
                    X = data['X']  # (C, H, W)

                    # CONFIRMED: SO2: mask==1 is valid pixels, mask==0 is invalid
                    # Additional robustness: ensure target > 0 for physical validity
                    valid_mask = (data['mask'] == 1) & (data['y'] > 0)

                    if feature_order is None:
                        # Get feature names
                        if 'feature_order' in data.files:
                            feature_order = data['feature_order'].tolist()
                        elif 'feature_names' in data.files:
                            feature_order = data['feature_names'].tolist()
                        else:
                            print(f"⚠️ No feature names found: {fname}")
                            continue

                    # --- SO2 鲁棒性检查：验证特征矩阵与特征名一致性 ---
                    if X.shape[0] != len(feature_order):
                        print(f"⚠️ Feature matrix shape mismatch in {fname}: X.shape[0]={X.shape[0]}, feature_order length={len(feature_order)}")
                        # 可以选择跳过或调整，这里选择跳过
                        continue

                    # --- 关键修正：正确处理2D掩膜索引 ---
                    C, H, W = X.shape
                    X_flat = X.reshape(C, -1)  # (C, H*W) - 打平空间维度
                    valid_flat = valid_mask.ravel()  # (H*W,) - 打平掩膜
                    valid_pixels = X_flat[:, valid_flat]  # (C, N_valid) - 正确提取有效像素

                    if valid_pixels.shape[1] == 0:
                        continue

                    # Random sampling to avoid memory issues (using fixed seed for reproducibility)
                    n_samples = min(sample_per_file, valid_pixels.shape[1])
                    if n_samples < valid_pixels.shape[1]:
                        indices = rng.choice(valid_pixels.shape[1], n_samples, replace=False)
                        sampled = valid_pixels[:, indices]
                    else:
                        sampled = valid_pixels

                    # Filter NaN/inf
                    finite_mask = np.isfinite(sampled)

                    # Accumulate statistics
                    for j, feat_name in enumerate(feature_order):
                        if feat_name in CONTINUOUS_FEATURES:
                            values = sampled[j]
                            finite_values = values[finite_mask[j]]

                            if len(finite_values) > 0:
                                # 1-99 percentile winsorization
                                q1, q99 = np.percentile(finite_values, [1, 99])
                                clipped = np.clip(finite_values, q1, q99)

                                stats[feat_name]['sum'] += np.sum(clipped)
                                stats[feat_name]['sumsq'] += np.sum(clipped**2)
                                stats[feat_name]['count'] += len(clipped)
                        elif feat_name in CATEGORICAL_FEATURES:
                            # Categorical features don't need standardization
                            pass
                        elif feat_name in NON_STANDARDIZED:
                            # Non-standardized features don't need standardization
                            pass

                    processed_files += 1

            except Exception as e:
                print(f"❌ Failed to load file {fname}: {e}")
                continue

    print(f"\n📊 Processing statistics:")
    print(f"Total files: {total_files}")
    print(f"Successfully processed: {processed_files}")
    print(f"Processing rate: {processed_files/total_files*100:.1f}%")

    if not stats:
        print("❌ No valid samples found")
        return None

    # Compute final statistics
    scalers = {}
    print(f"\n🔢 Computing standardization parameters...")

    # --- 名字一致性自检 ---
    print(f"\n🔍 Feature name consistency check:")
    all_defined_features = set(CONTINUOUS_FEATURES + CATEGORICAL_FEATURES + NON_STANDARDIZED)
    feature_order_set = set(feature_order)

    missing_in_order = all_defined_features - feature_order_set
    extra_in_order = feature_order_set - all_defined_features

    if missing_in_order:
        print(f"  ⚠️ Features defined but missing in feature_order: {missing_in_order}")
    if extra_in_order:
        print(f"  ⚠️ Features in feature_order but not defined: {extra_in_order}")
    if not missing_in_order and not extra_in_order:
        print(f"  ✅ All feature names are consistent!")

    # 检查关键字段名
    if 'ssr_clear' in feature_order:
        print(f"  ✅ SO2 uses correct field name: ssr_clear")
    else:
        print(f"  ❌ Missing ssr_clear in feature_order")

    for feat_name in feature_order:
        if feat_name in CONTINUOUS_FEATURES:
            # 连续特征：统一处理，即使没有统计到也给默认值
            if feat_name in stats:
                count = stats[feat_name]['count']
                if count > 0:
                    mean = stats[feat_name]['sum'] / count
                    variance = (stats[feat_name]['sumsq'] / count) - (mean**2)
                    std = np.sqrt(max(variance, 1e-8))  # Avoid division by zero

                    scalers[feat_name] = {
                        'mean': float(mean),
                        'std': float(std),
                        'count': int(count)
                    }
                    print(f"  {feat_name}: mean={mean:.4f}, std={std:.4f}, count={count}")
                else:
                    # 有统计但count=0，给默认值
                    scalers[feat_name] = {'mean': 0.0, 'std': 1.0, 'count': 0}
                    print(f"  {feat_name}: No valid samples, using default values")
            else:
                # 完全没有统计到，也给默认值
                scalers[feat_name] = {'mean': 0.0, 'std': 1.0, 'count': 0}
                print(f"  {feat_name}: No statistics collected, using default values")
        elif feat_name in CATEGORICAL_FEATURES:
            scalers[feat_name] = {'type': 'categorical'}
            print(f"  {feat_name}: categorical (one-hot)")
        elif feat_name in NON_STANDARDIZED:
            scalers[feat_name] = {'type': 'non_standardized'}
            print(f"  {feat_name}: non_standardized")
        else:
            # 这种情况理论上不应该发生，因为feature_order应该包含所有特征
            scalers[feat_name] = {'type': 'unexpected'}
            print(f"  {feat_name}: unexpected type - check feature_order definition")

    return scalers, feature_order

def apply_so2_scaler(X: np.ndarray, feature_names: list, scaler_file: str = None):
    """
    Apply SO2 standardization to feature matrix

    Args:
        X: Feature matrix (C, H, W) or (C, N)
        feature_names: List of feature names
        scaler_file: Path to scaler file (if None, auto-detect)

    Returns:
        X_scaled: Standardized feature matrix
    """
    if scaler_file is None:
        scaler_file = os.path.join(OUT_DIR, "so2_scalers_2019_2021.npz")

    if not os.path.exists(scaler_file):
        raise FileNotFoundError(f"SO2 scaler file not found: {scaler_file}")

    with np.load(scaler_file, allow_pickle=True) as data:
        scalers = data['scalers'].item()

    X_scaled = X.copy().astype(np.float32)

    for i, feat_name in enumerate(feature_names):
        if feat_name in scalers and 'mean' in scalers[feat_name]:
            # Apply z-score normalization for continuous features
            mean = scalers[feat_name]['mean']
            std = scalers[feat_name]['std']
            X_scaled[i] = (X[i] - mean) / std
        # Skip categorical and non-standardized features (they remain unchanged)

    return X_scaled

# Compute SO2 standardization parameters
print("=" * 60)
print("🚀 SO2 Standardization Parameter Computation")
print("=" * 60)

so2_scalers, so2_features = compute_so2_scalers()

if so2_scalers is not None:
    # Save results
    output_file = os.path.join(OUT_DIR, "so2_scalers_2019_2021.npz")
    np.savez_compressed(
        output_file,
        scalers=so2_scalers,
        feature_order=so2_features,
        metadata={
            'data_type': 'so2',
            'years': [2019, 2020, 2021],
            'mask_logic': 'mask==1 is valid, mask==0 is invalid (CONFIRMED FROM SOURCE CODE)',
            'total_features': len(so2_features),
            'continuous_features': len([f for f in so2_features if f in CONTINUOUS_FEATURES])
        }
    )

    print(f"\n✅ SO2 standardization parameters saved to: {output_file}")
    print(f" Feature statistics:")
    print(f"  Total features: {len(so2_features)}")
    print(f"  Continuous features: {len([f for f in so2_features if f in CONTINUOUS_FEATURES])}")
    print(f"  Categorical features: {len([f for f in so2_features if f in CATEGORICAL_FEATURES])}")
    print(f"  Non-standardized features: {len([f for f in so2_features if f in NON_STANDARDIZED])}")

    # Show parameters for first few continuous features
    print(f"\n🔍 Standardization parameters for first 5 continuous features:")
    continuous_count = 0
    for feat_name in so2_features:
        if feat_name in CONTINUOUS_FEATURES and feat_name in so2_scalers:
            if 'mean' in so2_scalers[feat_name]:
                print(f"  {feat_name}: mean={so2_scalers[feat_name]['mean']:.4f}, std={so2_scalers[feat_name]['std']:.4f}")
                continuous_count += 1
                if continuous_count >= 5:
                    break
else:
    print("❌ Computation failed")

print("\n Next step: Ready to train SO2 model!")

🚀 SO2 Standardization Parameter Computation
🚀 Starting SO2 standardization parameter computation...
Data path: /content/drive/MyDrive/Feature_Stacks
Training years: [2019, 2020, 2021]
Samples per file: 1000

 Processing 2019: 365 files
  0/365 (0.0%)
  50/365 (13.7%)
  100/365 (27.4%)
  150/365 (41.1%)
  200/365 (54.8%)
  250/365 (68.5%)
  300/365 (82.2%)
  350/365 (95.9%)

 Processing 2020: 366 files
  0/366 (0.0%)
  50/366 (13.7%)
  100/366 (27.3%)
  150/366 (41.0%)
  200/366 (54.6%)
  250/366 (68.3%)
  300/366 (82.0%)
  350/366 (95.6%)

 Processing 2021: 365 files
  0/365 (0.0%)
  50/365 (13.7%)
  100/365 (27.4%)
  150/365 (41.1%)
  200/365 (54.8%)
  250/365 (68.5%)
  300/365 (82.2%)
  350/365 (95.9%)

📊 Processing statistics:
Total files: 1096
Successfully processed: 779
Processing rate: 71.1%

🔢 Computing standardization parameters...

🔍 Feature name consistency check:
  ✅ All feature names are consistent!
  ✅ SO2 uses correct field name: ssr_clear
  dem: mean=400.9618, std=533.48

In [ ]:
# Quick Check SO2 Scaler Results
import os
import numpy as np

def quick_check_so2_scaler_results():
    """快速检查SO2 scaler结果"""
    print("🔍 Quick SO2 Scaler Results Check")
    print("=" * 50)

    # 检查scaler文件
    scaler_file = "/content/drive/MyDrive/Scalers/so2_scalers_2019_2021.npz"

    if not os.path.exists(scaler_file):
        print("❌ SO2 scaler file not found")
        return

    print("✅ Scaler file exists")

    try:
        with np.load(scaler_file, allow_pickle=True) as data:
            print("✅ File loaded successfully")

            # 检查基本结构
            print(f"📊 Available keys: {list(data.files)}")

            # 检查元数据
            if 'metadata' in data.files:
                metadata = data['metadata'].item()
                print(f"📅 Years: {metadata.get('years', 'Unknown')}")
                print(f"📁 Total files: {metadata.get('total_files', 'Unknown')}")
                print(f"🎯 Success rate: {metadata.get('success_rate', 'Unknown')}")
                print(f"📊 Total samples: {metadata.get('total_samples', 'Unknown')}")

            # 检查scaler参数
            if 'scalers' in data.files:
                scalers = data['scalers'].item()
                print(f"📊 Total features: {len(scalers)}")

                # 分析连续特征
                continuous_features = []
                categorical_features = []
                non_standardized_features = []

                for feat_name, feat_info in scalers.items():
                    if isinstance(feat_info, dict):
                        if 'mean' in feat_info and 'std' in feat_info:
                            continuous_features.append(feat_name)
                        elif 'categorical' in feat_info:
                            categorical_features.append(feat_name)
                        else:
                            non_standardized_features.append(feat_name)

                print(f"📊 Feature type summary:")
                print(f"  Continuous features: {len(continuous_features)}")
                print(f"  Categorical features: {len(categorical_features)}")
                print(f"  Non-standardized features: {len(non_standardized_features)}")

                # 检查关键连续特征
                print(f"\n🎯 Key Continuous Features Analysis:")
                key_features = ['dem', 'slope', 'population', 'u10', 'v10', 'blh', 'tp', 't2m', 'sp', 'str', 'ssr_clear', 'ws', 'so2_lag1', 'so2_neighbor', 'so2_climate_prior']

                for feat_name in key_features:
                    if feat_name in scalers and 'mean' in scalers[feat_name]:
                        mean = scalers[feat_name]['mean']
                        std = scalers[feat_name]['std']
                        count = scalers[feat_name].get('count', 'Unknown')
                        print(f"  ✅ {feat_name}: {count:,} samples, mean={mean:.4f}, std={std:.4f}")
                    else:
                        print(f"  ❌ {feat_name}: Not found or no statistics")

                # 检查问题特征
                print(f"\n🔍 Problem Feature Check:")
                problem_features = ['sp', 'str', 'ssr_clear']

                for feat_name in problem_features:
                    if feat_name in scalers and 'mean' in scalers[feat_name]:
                        mean = scalers[feat_name]['mean']
                        std = scalers[feat_name]['std']
                        count = scalers[feat_name].get('count', 'Unknown')

                        if abs(mean) < 1e-6 and abs(std) < 1e-6:
                            print(f"  ⚠️ {feat_name}: Near-zero values (mean={mean:.6f}, std={std:.6f})")
                        elif count == 0:
                            print(f"  ⚠️ {feat_name}: No samples")
                        else:
                            print(f"  ✅ {feat_name}: Normal values (mean={mean:.4f}, std={std:.4f})")
                    else:
                        print(f"  ❌ {feat_name}: Not found")

                # 计算成功率
                total_continuous = len(continuous_features)
                valid_continuous = sum(1 for f in key_features if f in scalers and 'mean' in scalers[f])
                success_rate = valid_continuous / len(key_features) * 100

                print(f"\n🎯 Success Rate: {success_rate:.1f}%")

                if success_rate >= 90:
                    print("🎉 SO2 Scaler looks excellent!")
                elif success_rate >= 80:
                    print("✅ SO2 Scaler looks good!")
                elif success_rate >= 70:
                    print("⚠️ SO2 Scaler has some issues")
                else:
                    print("❌ SO2 Scaler has significant issues")

                # 检查数据质量
                print(f"\n📊 Data Quality Assessment:")

                # 检查样本数量
                sample_counts = [scalers[f].get('count', 0) for f in continuous_features if f in scalers and 'count' in scalers[f]]
                if sample_counts:
                    min_samples = min(sample_counts)
                    max_samples = max(sample_counts)
                    avg_samples = sum(sample_counts) / len(sample_counts)

                    print(f"  Sample count range: {min_samples:,} - {max_samples:,}")
                    print(f"  Average samples: {avg_samples:,.0f}")

                    if min_samples < 1000:
                        print(f"  ⚠️ Some features have very low sample counts")
                    elif max_samples - min_samples > max_samples * 0.1:
                        print(f"  ⚠️ Sample count variation is high")
                    else:
                        print(f"  ✅ Sample counts are consistent")

                # 检查数值范围
                print(f"\n🔍 Value Range Analysis:")
                for feat_name in ['sp', 'str', 'ssr_clear']:
                    if feat_name in scalers and 'mean' in scalers[feat_name]:
                        mean = scalers[feat_name]['mean']
                        std = scalers[feat_name]['std']

                        if feat_name == 'sp':
                            if 90000 < mean < 110000:
                                print(f"  ✅ {feat_name}: Pressure values look normal")
                            else:
                                print(f"  ⚠️ {feat_name}: Pressure values may be unusual")
                        elif feat_name == 'str':
                            if mean < 0:
                                print(f"  ✅ {feat_name}: Thermal radiation values look normal (negative)")
                            else:
                                print(f"  ⚠️ {feat_name}: Thermal radiation values may be unusual")
                        elif feat_name == 'ssr_clear':
                            if 0 < mean < 10000000:
                                print(f"  ✅ {feat_name}: Solar radiation values look normal")
                            else:
                                print(f"  ⚠️ {feat_name}: Solar radiation values may be unusual")

    except Exception as e:
        print(f"❌ Error loading scaler file: {e}")

if __name__ == "__main__":
    quick_check_so2_scaler_results()


🔍 Quick SO2 Scaler Results Check
✅ Scaler file exists
✅ File loaded successfully
📊 Available keys: ['scalers', 'feature_order', 'metadata']
📅 Years: [2019, 2020, 2021]
📁 Total files: Unknown
🎯 Success rate: Unknown
📊 Total samples: Unknown
📊 Total features: 30
📊 Feature type summary:
  Continuous features: 15
  Categorical features: 0
  Non-standardized features: 15

🎯 Key Continuous Features Analysis:
  ✅ dem: 761,779 samples, mean=400.9618, std=533.4883
  ✅ slope: 761,779 samples, mean=9.2690, std=9.5274
  ✅ population: 761,779 samples, mean=145.4318, std=384.2961
  ✅ u10: 761,779 samples, mean=-0.4001, std=1.7508
  ✅ v10: 761,779 samples, mean=0.2341, std=1.5989
  ✅ blh: 761,779 samples, mean=1191.6626, std=466.9831
  ✅ tp: 761,779 samples, mean=0.0003, std=0.0010
  ✅ t2m: 761,779 samples, mean=20.3405, std=6.8925
  ✅ sp: 761,779 samples, mean=96270.7153, std=5620.0365
  ✅ str: 761,779 samples, mean=-1467077.4248, std=368888.7481
  ✅ ssr_clear: 761,779 samples, mean=8178523.0930, st

In [ ]:
# Validate SO2 Scaler Results
import os
import numpy as np
import matplotlib.pyplot as plt

def validate_so2_scaler_results():
    """验证SO2 scaler结果"""
    print("🔍 SO2 Scaler Results Validation")
    print("=" * 50)

    # 检查scaler文件
    scaler_file = "/content/drive/MyDrive/Scalers/so2_scalers_2019_2021.npz"

    if not os.path.exists(scaler_file):
        print("❌ SO2 scaler file not found")
        return

    print("✅ Scaler file exists")

    try:
        with np.load(scaler_file, allow_pickle=True) as data:
            print("✅ File loaded successfully")

            # 检查基本结构
            print(f"📊 Available keys: {list(data.files)}")

            # 检查元数据
            if 'metadata' in data.files:
                metadata = data['metadata'].item()
                print(f"\n📋 Metadata:")
                print(f"  Years: {metadata.get('years', 'Unknown')}")
                print(f"  Total files: {metadata.get('total_files', 'Unknown')}")
                print(f"  Success rate: {metadata.get('success_rate', 'Unknown')}")
                print(f"  Total samples: {metadata.get('total_samples', 'Unknown')}")
                print(f"  Processing time: {metadata.get('processing_time', 'Unknown')}")
                print(f"  Mask logic: {metadata.get('mask_logic', 'Unknown')}")

            # 检查scaler参数
            if 'scalers' in data.files:
                scalers = data['scalers'].item()
                print(f"\n📊 Scaler Parameters:")
                print(f"  Total features: {len(scalers)}")

                # 分析特征类型
                continuous_features = []
                categorical_features = []
                non_standardized_features = []

                for feat_name, feat_info in scalers.items():
                    if isinstance(feat_info, dict):
                        if 'mean' in feat_info and 'std' in feat_info:
                            continuous_features.append(feat_name)
                        elif 'categorical' in feat_info:
                            categorical_features.append(feat_name)
                        else:
                            non_standardized_features.append(feat_name)

                print(f"  Continuous features: {len(continuous_features)}")
                print(f"  Categorical features: {len(categorical_features)}")
                print(f"  Non-standardized features: {len(non_standardized_features)}")

                # 详细分析连续特征
                print(f"\n🔍 Continuous Features Analysis:")
                for feat_name in continuous_features:
                    if feat_name in scalers:
                        feat_info = scalers[feat_name]
                        mean = feat_info.get('mean', 0)
                        std = feat_info.get('std', 0)
                        count = feat_info.get('count', 0)

                        print(f"  {feat_name}:")
                        print(f"    Mean: {mean:.6f}")
                        print(f"    Std: {std:.6f}")
                        print(f"    Count: {count:,}")

                        # 检查数据质量
                        if count == 0:
                            print(f"    ⚠️ No samples")
                        elif abs(mean) < 1e-6 and abs(std) < 1e-6:
                            print(f"    ⚠️ Near-zero values")
                        elif std == 0:
                            print(f"    ⚠️ Zero standard deviation")
                        else:
                            print(f"    ✅ Normal statistics")

                # 检查关键特征
                print(f"\n🎯 Key Features Check:")
                key_features = {
                    'dem': 'Elevation (m)',
                    'slope': 'Slope (degrees)',
                    'population': 'Population density',
                    'u10': 'U-wind component (m/s)',
                    'v10': 'V-wind component (m/s)',
                    'blh': 'Boundary layer height (m)',
                    'tp': 'Total precipitation (m)',
                    't2m': 'Temperature (K)',
                    'sp': 'Surface pressure (Pa)',
                    'str': 'Surface thermal radiation (J/m²)',
                    'ssr_clear': 'Clear-sky solar radiation (J/m²)',
                    'ws': 'Wind speed (m/s)',
                    'so2_lag1': 'SO2 lag-1 day',
                    'so2_neighbor': 'SO2 neighbor mean',
                    'so2_climate_prior': 'SO2 climate prior'
                }

                for feat_name, description in key_features.items():
                    if feat_name in scalers and 'mean' in scalers[feat_name]:
                        mean = scalers[feat_name]['mean']
                        std = scalers[feat_name]['std']
                        count = scalers[feat_name].get('count', 0)

                        print(f"  ✅ {feat_name} ({description}):")
                        print(f"    Mean: {mean:.4f}")
                        print(f"    Std: {std:.4f}")
                        print(f"    Samples: {count:,}")

                        # 物理合理性检查
                        if feat_name == 'sp' and (mean < 80000 or mean > 110000):
                            print(f"    ⚠️ Pressure values may be unusual")
                        elif feat_name == 't2m' and (mean < 250 or mean > 320):
                            print(f"    ⚠️ Temperature values may be unusual")
                        elif feat_name == 'str' and mean > 0:
                            print(f"    ⚠️ Thermal radiation should be negative")
                        elif feat_name == 'ssr_clear' and (mean < 0 or mean > 20000000):
                            print(f"    ⚠️ Solar radiation values may be unusual")
                    else:
                        print(f"  ❌ {feat_name}: Not found")

                # 计算质量评分
                print(f"\n📊 Quality Assessment:")

                # 检查样本数量一致性
                sample_counts = [scalers[f].get('count', 0) for f in continuous_features if f in scalers and 'count' in scalers[f]]
                if sample_counts:
                    min_samples = min(sample_counts)
                    max_samples = max(sample_counts)
                    avg_samples = sum(sample_counts) / len(sample_counts)

                    print(f"  Sample count analysis:")
                    print(f"    Min: {min_samples:,}")
                    print(f"    Max: {max_samples:,}")
                    print(f"    Average: {avg_samples:,.0f}")
                    print(f"    Variation: {((max_samples - min_samples) / avg_samples * 100):.1f}%")

                    if (max_samples - min_samples) / avg_samples < 0.05:
                        print(f"    ✅ Sample counts are very consistent")
                    elif (max_samples - min_samples) / avg_samples < 0.1:
                        print(f"    ✅ Sample counts are consistent")
                    else:
                        print(f"    ⚠️ Sample counts vary significantly")

                # 检查数值范围
                print(f"\n🔍 Value Range Analysis:")
                for feat_name in ['sp', 'str', 'ssr_clear']:
                    if feat_name in scalers and 'mean' in scalers[feat_name]:
                        mean = scalers[feat_name]['mean']
                        std = scalers[feat_name]['std']

                        if feat_name == 'sp':
                            if 90000 < mean < 110000:
                                print(f"  ✅ {feat_name}: Pressure values are in normal range")
                            else:
                                print(f"  ⚠️ {feat_name}: Pressure values may be unusual")
                        elif feat_name == 'str':
                            if mean < 0:
                                print(f"  ✅ {feat_name}: Thermal radiation values are normal (negative)")
                            else:
                                print(f"  ⚠️ {feat_name}: Thermal radiation values may be unusual")
                        elif feat_name == 'ssr_clear':
                            if 0 < mean < 20000000:
                                print(f"  ✅ {feat_name}: Solar radiation values are in normal range")
                            else:
                                print(f"  ⚠️ {feat_name}: Solar radiation values may be unusual")

                # 总体质量评分
                quality_score = calculate_quality_score(scalers, continuous_features)
                print(f"\n🎯 Overall Quality Score: {quality_score:.1f}/100")

                if quality_score >= 90:
                    print("🎉 Excellent quality! Ready for training.")
                elif quality_score >= 80:
                    print("✅ Good quality! Ready for training.")
                elif quality_score >= 70:
                    print("⚠️ Acceptable quality. Consider reviewing some features.")
                else:
                    print("❌ Poor quality. Significant issues need to be addressed.")

    except Exception as e:
        print(f"❌ Error loading scaler file: {e}")

def calculate_quality_score(scalers, continuous_features):
    """计算质量评分"""
    score = 100.0

    # 检查连续特征
    for feat_name in continuous_features:
        if feat_name in scalers:
            feat_info = scalers[feat_name]
            count = feat_info.get('count', 0)
            mean = feat_info.get('mean', 0)
            std = feat_info.get('std', 0)

            # 样本数量检查
            if count == 0:
                score -= 10
            elif count < 1000:
                score -= 5

            # 数值合理性检查
            if abs(mean) < 1e-6 and abs(std) < 1e-6:
                score -= 15
            elif std == 0:
                score -= 10

            # 物理合理性检查
            if feat_name == 'sp' and (mean < 80000 or mean > 110000):
                score -= 5
            elif feat_name == 't2m' and (mean < 250 or mean > 320):
                score -= 5
            elif feat_name == 'str' and mean > 0:
                score -= 5
            elif feat_name == 'ssr_clear' and (mean < 0 or mean > 20000000):
                score -= 5

    return max(0, score)

if __name__ == "__main__":
    validate_so2_scaler_results()


🔍 SO2 Scaler Results Validation
✅ Scaler file exists
✅ File loaded successfully
📊 Available keys: ['scalers', 'feature_order', 'metadata']

📋 Metadata:
  Years: [2019, 2020, 2021]
  Total files: Unknown
  Success rate: Unknown
  Total samples: Unknown
  Processing time: Unknown
  Mask logic: mask==1 is valid, mask==0 is invalid (CONFIRMED FROM SOURCE CODE)

📊 Scaler Parameters:
  Total features: 30
  Continuous features: 15
  Categorical features: 0
  Non-standardized features: 15

🔍 Continuous Features Analysis:
  dem:
    Mean: 400.961806
    Std: 533.488280
    Count: 761,779
    ✅ Normal statistics
  slope:
    Mean: 9.268992
    Std: 9.527353
    Count: 761,779
    ✅ Normal statistics
  population:
    Mean: 145.431770
    Std: 384.296097
    Count: 761,779
    ✅ Normal statistics
  u10:
    Mean: -0.400131
    Std: 1.750753
    Count: 761,779
    ✅ Normal statistics
  v10:
    Mean: 0.234117
    Std: 1.598915
    Count: 761,779
    ✅ Normal statistics
  ws:
    Mean: 2.022176
   

In [ ]:
# 检查实际文件状态
import os

def check_manifest_status():
    """检查SO2和NO2的manifest文件状态"""
    print("🔍 检查manifest文件状态...")

    # 检查SO2
    so2_manifest_paths = [
        "/content/drive/MyDrive/Feature_Stacks/so2_feature_stacks/so2_feature_stacks_manifest.csv",
        "/content/drive/MyDrive/so2_train_20250911_080237.txt",
        "/content/drive/MyDrive/so2_val_20250911_080237.txt",
        "/content/drive/MyDrive/so2_test_20250911_080237.txt"
    ]

    print("\n📊 SO2文件状态:")
    for path in so2_manifest_paths:
        exists = os.path.exists(path)
        print(f"  {path}: {'✅' if exists else '❌'}")

    # 检查NO2
    no2_manifest_paths = [
        "/content/drive/MyDrive/Feature_Stacks/no2_feature_stacks/no2_feature_stacks_manifest.csv",
        "/content/drive/MyDrive/no2_train_*.txt",
        "/content/drive/MyDrive/no2_val_*.txt",
        "/content/drive/MyDrive/no2_test_*.txt"
    ]

    print("\n NO2文件状态:")
    for path in no2_manifest_paths:
        if "*" in path:
            # 检查通配符文件
            import glob
            files = glob.glob(path)
            print(f"  {path}: {'✅' if files else '❌'} ({len(files)} files)")
        else:
            exists = os.path.exists(path)
            print(f"  {path}: {'✅' if exists else '❌'}")

# 运行检查
check_manifest_status()

🔍 检查manifest文件状态...

📊 SO2文件状态:
  /content/drive/MyDrive/Feature_Stacks/so2_feature_stacks/so2_feature_stacks_manifest.csv: ❌
  /content/drive/MyDrive/so2_train_20250911_080237.txt: ✅
  /content/drive/MyDrive/so2_val_20250911_080237.txt: ✅
  /content/drive/MyDrive/so2_test_20250911_080237.txt: ✅

 NO2文件状态:
  /content/drive/MyDrive/Feature_Stacks/no2_feature_stacks/no2_feature_stacks_manifest.csv: ❌
  /content/drive/MyDrive/no2_train_*.txt: ❌ (0 files)
  /content/drive/MyDrive/no2_val_*.txt: ❌ (0 files)
  /content/drive/MyDrive/no2_test_*.txt: ❌ (0 files)


In [ ]:
# 修正版：生成SO2和NO2的manifest文件（符合最佳实践）
import os
import pandas as pd
import numpy as np
import re
from datetime import datetime
import glob

BASE_SO2 = "/content/drive/MyDrive/Variables/so2_feature_stacks"
BASE_NO2 = "/content/drive/MyDrive/Variables/no2_feature_stacks"
OUT_DIR = "/content/drive/MyDrive"

def create_manifest_improved(data_type, base_path, output_dir):
    """创建manifest文件（改进版）"""
    print(f"\n📋 创建{data_type} manifest文件（改进版）...")

    manifest_data = []
    coverage_issues = []

    for year in [2019, 2020, 2021, 2022, 2023]:
        year_dir = os.path.join(base_path, str(year))
        if not os.path.exists(year_dir):
            print(f"⚠️ 年份目录不存在: {year_dir}")
            continue

        files = [f for f in os.listdir(year_dir) if f.endswith('.npz')]
        files.sort()

        print(f"  {year}: {len(files)} 个文件")

        for file in files:
            try:
                # 改进1: 更robust的日期解析
                m = re.search(r'(\d{8})', file)
                if not m:
                    print(f"⚠️ 无法解析日期: {file}")
                    continue

                date8 = m.group(1)
                date_formatted = f"{date8[:4]}-{date8[4:6]}-{date8[6:8]}"

                file_path = os.path.join(year_dir, file)

                # 改进3: 覆盖率一致性校验
                try:
                    with np.load(file_path, allow_pickle=True) as d:
                        # 基于mask重新计算覆盖率
                        if 'mask' in d.files:
                            cov_chk = float((d['mask']==0).sum())/d['mask'].size
                        else:
                            cov_chk = np.nan

                        # 使用文件中的coverage，如果没有则使用计算值
                        coverage = float(d['coverage']) if 'coverage' in d.files else cov_chk

                        # 一致性检查
                        if 'coverage' in d.files and not np.isnan(cov_chk):
                            diff = abs(coverage - cov_chk)
                            if diff > 0.01:  # 差异超过1%
                                coverage_issues.append({
                                    'file': file,
                                    'stored_coverage': coverage,
                                    'calculated_coverage': cov_chk,
                                    'difference': diff
                                })

                        trainable = bool(d['trainable']) if 'trainable' in d.files else False
                        season = d['season'].item() if 'season' in d.files else 'unknown'

                except Exception as e:
                    print(f"⚠️ 加载文件失败 {file}: {e}")
                    continue

                manifest_data.append({
                    'date': date_formatted,
                    'path': file_path,
                    'coverage': coverage,
                    'trainable': trainable,
                    'season': season,
                    'year': int(date8[:4]),
                    'month': int(date8[4:6]),
                    'day': int(date8[6:8])
                })

            except Exception as e:
                print(f"⚠️ 处理文件失败 {file}: {e}")
                continue

    # 创建DataFrame并保存
    manifest_df = pd.DataFrame(manifest_data)
    manifest_file = os.path.join(output_dir, f'{data_type.lower()}_manifest.csv')
    manifest_df.to_csv(manifest_file, index=False)

    print(f"✅ {data_type} manifest保存: {manifest_file}")
    print(f"   总文件数: {len(manifest_df)}")
    print(f"   日期范围: {manifest_df['date'].min()} 到 {manifest_df['date'].max()}")
    print(f"   可训练文件: {manifest_df['trainable'].sum()}")

    # 报告覆盖率问题
    if coverage_issues:
        print(f"⚠️ 发现 {len(coverage_issues)} 个覆盖率不一致文件")
        for issue in coverage_issues[:5]:  # 显示前5个
            print(f"   {issue['file']}: 存储={issue['stored_coverage']:.3f}, 计算={issue['calculated_coverage']:.3f}, 差异={issue['difference']:.3f}")

    return manifest_df

def create_train_val_test_splits_improved(manifest_df, data_type, output_dir):
    """创建train/val/test划分（改进版）"""
    print(f"\n 创建{data_type} train/val/test划分（改进版）...")

    # 过滤可训练文件
    trainable_df = manifest_df[manifest_df['trainable'] == True].copy()
    print(f"   可训练文件: {len(trainable_df)}")

    if len(trainable_df) == 0:
        print(f"❌ 没有可训练文件")
        return

    # 按年份划分
    train_files = trainable_df[trainable_df['year'].isin([2019, 2020, 2021])]
    val_files = trainable_df[trainable_df['year'] == 2022]
    test_files = trainable_df[trainable_df['year'] == 2023]

    print(f"   训练集: {len(train_files)} 文件")
    print(f"   验证集: {len(val_files)} 文件")
    print(f"   测试集: {len(test_files)} 文件")

    # 保存划分文件
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    for split_name, split_df in [('train', train_files), ('val', val_files), ('test', test_files)]:
        if len(split_df) > 0:
            # 改进2: 同时保存TXT和CSV文件
            # TXT文件（仅日期，兼容现有代码）
            split_txt = os.path.join(output_dir, f'{data_type.lower()}_{split_name}_{timestamp}.txt')
            with open(split_txt, 'w') as f:
                for _, row in split_df.iterrows():
                    f.write(f"{row['date']}\n")

            # CSV文件（包含完整元数据）
            split_csv = os.path.join(output_dir, f'{data_type.lower()}_{split_name}_{timestamp}.csv')
            split_df[['date', 'path', 'coverage', 'season']].to_csv(split_csv, index=False)

            print(f"   ✅ {split_name}: {split_txt} + {split_csv}")

    return train_files, val_files, test_files

def main():
    """主函数"""
    print(" 生成SO2和NO2的manifest文件以及train/val/test划分（改进版）...")

    # 创建SO2 manifest和划分
    if os.path.exists(BASE_SO2):
        so2_manifest = create_manifest_improved("SO2", BASE_SO2, OUT_DIR)
        create_train_val_test_splits_improved(so2_manifest, "SO2", OUT_DIR)
    else:
        print(f"❌ SO2数据目录不存在: {BASE_SO2}")

    # 创建NO2 manifest和划分
    if os.path.exists(BASE_NO2):
        no2_manifest = create_manifest_improved("NO2", BASE_NO2, OUT_DIR)
        create_train_val_test_splits_improved(no2_manifest, "NO2", OUT_DIR)
    else:
        print(f"❌ NO2数据目录不存在: {BASE_NO2}")

    print("\n✅ 所有manifest和划分文件生成完成！")

# 运行主函数
main()

 生成SO2和NO2的manifest文件以及train/val/test划分（改进版）...

📋 创建SO2 manifest文件（改进版）...
  2019: 365 个文件
  2020: 366 个文件
  2021: 365 个文件
  2022: 365 个文件
  2023: 365 个文件
✅ SO2 manifest保存: /content/drive/MyDrive/so2_manifest.csv
   总文件数: 1826
   日期范围: 2019-01-01 到 2023-12-31
   可训练文件: 852
⚠️ 发现 1024 个覆盖率不一致文件
   so2_features_20190206.npz: 存储=0.012, 计算=0.023, 差异=0.011
   so2_features_20190207.npz: 存储=0.015, 计算=0.046, 差异=0.031
   so2_features_20190211.npz: 存储=0.050, 计算=0.097, 差异=0.047
   so2_features_20190212.npz: 存储=0.076, 计算=0.239, 差异=0.164
   so2_features_20190213.npz: 存储=0.048, 计算=0.169, 差异=0.121

 创建SO2 train/val/test划分（改进版）...
   可训练文件: 852
   训练集: 507 文件
   验证集: 175 文件
   测试集: 170 文件
   ✅ train: /content/drive/MyDrive/so2_train_20250912_083216.txt + /content/drive/MyDrive/so2_train_20250912_083216.csv
   ✅ val: /content/drive/MyDrive/so2_val_20250912_083216.txt + /content/drive/MyDrive/so2_val_20250912_083216.csv
   ✅ test: /content/drive/MyDrive/so2_test_20250912_083216.txt + /content/drive/MyD

In [ ]:
# Unified manifest + split builder (with per-dataset thresholds and QC guards)
import os, re, numpy as np, pandas as pd, random
from datetime import datetime

# ----------------------------- Config -----------------------------
BASE_NO2 = "/content/drive/MyDrive/Variables/no2_feature_stacks"
BASE_SO2 = "/content/drive/MyDrive/Variables/so2_feature_stacks"
OUT_DIR   = "/content/drive/MyDrive/Variables"
YEARS     = [2019, 2020, 2021, 2022, 2023]

# Year-based splits
TRAIN_YRS, VAL_YRS, TEST_YRS = [2019, 2020, 2021], [2022], [2023]

# Per-dataset default trainable thresholds (overridable by file field 'trainable_threshold')
DEFAULT_THR = {"NO2": 0.40, "SO2": 0.10}

# QC guards only when coverage_source == 'mask'
MIN_YPOS_FRAC = 0.01     # require at least 1% positive y pixels globally (very loose)
MAX_COV_DIFF  = 0.05     # |mean(mask==0) - mean(y>0 & finite)| should be < 0.05

# Repro
random.seed(42); np.random.seed(42)

# ----------------------------- Helpers -----------------------------
def parse_date_from_filename(fname: str) -> str | None:
    m = re.search(r'(\d{8})', fname)
    if not m:
        return None
    d = m.group(1)
    return f"{d[:4]}-{d[4:6]}-{d[6:8]}"

def compute_coverage_unified(fp: str, data_type: str):
    """
    Returns:
      coverage: float
      trainable: bool (after applying threshold and QC if needed)
      threshold_used: float
      season: str
      coverage_source: str in {'domain+data','data_coverage','mask','none','error'}
      y_positive_frac: float (only meaningful when source=='mask', else np.nan)
      qc_pass: bool
    """
    try:
        with np.load(fp, allow_pickle=True) as d:
            # 1) coverage compute with unified precedence
            if {'data_mask','domain_mask'}.issubset(d.files):
                source = 'domain+data'
                dom = (d['domain_mask'] == 1)
                dat = (d['data_mask'] == 1)
                denom = max(dom.sum(), 1)
                cov = float((dat & dom).sum() / denom)
            elif 'data_coverage' in d.files:
                source = 'data_coverage'
                cov = float(d['data_coverage'])
            elif 'mask' in d.files:
                source = 'mask'
                cov = float((d['mask']==0).sum() / d['mask'].size)
            else:
                source = 'none'
                cov = 0.0

            # 2) threshold
            thr = float(d['trainable_threshold']) if 'trainable_threshold' in d.files else DEFAULT_THR[data_type]
            season = d['season'].item() if 'season' in d.files else 'unknown'

            # 3) QC guard if we fell back to 'mask' (no AOI info)
            qc_pass = True
            y_pos_frac = np.nan
            if source == 'mask':
                try:
                    y = d['y']
                    mask = d['mask']
                    pos = np.isfinite(y) & (y > 0)
                    y_pos_frac = float(pos.sum() / y.size)
                    cov_mask = float((mask==0).sum() / mask.size)
                    qc_pass = (y_pos_frac >= MIN_YPOS_FRAC) and (abs(cov_mask - y_pos_frac) < MAX_COV_DIFF)
                except Exception:
                    qc_pass = False

            trainable = bool((cov >= thr) and qc_pass)
            return cov, trainable, thr, season, source, y_pos_frac, qc_pass

    except Exception:
        return 0.0, False, DEFAULT_THR[data_type], 'unknown', 'error', np.nan, False

def quick_summary(df: pd.DataFrame, name: str):
    if df.empty:
        print(f"\n[{name}] files=0")
        return
    print(f"\n[{name}] files={len(df)}  cov_mean={df['coverage'].mean():.3f}  cov_med={df['coverage'].median():.3f}")
    print(" season counts:\n", df['season'].value_counts())
    print(" coverage_source breakdown:\n", df['coverage_source'].value_counts())

def build_manifest(data_type: str, base_dir: str) -> pd.DataFrame:
    print(f"\n📋 Building manifest for {data_type} ...")
    rows = []
    for y in YEARS:
        ydir = os.path.join(base_dir, str(y))
        if not os.path.isdir(ydir):
            continue
        files = sorted([f for f in os.listdir(ydir) if f.endswith(".npz")])
        print(f"  {y}: {len(files)} files")
        for f in files:
            date = parse_date_from_filename(f)
            if not date:
                continue
            fp = os.path.join(ydir, f)
            cov, trn, thr, season, source, ypf, qc = compute_coverage_unified(fp, data_type)
            rows.append({
                'date': date,
                'path': fp,
                'coverage': cov,
                'trainable': trn,
                'threshold': thr,
                'season': season,
                'coverage_source': source,
                'y_positive_frac': ypf,
                'qc_pass': qc,
                'year': int(date[:4]),
                'month': int(date[5:7]),
                'day': int(date[8:10]),
            })
    df = pd.DataFrame(rows).sort_values('date').reset_index(drop=True)
    out_csv = os.path.join(OUT_DIR, f"{data_type.lower()}_manifest.csv")
    df.to_csv(out_csv, index=False)
    print(f"✅ {data_type} manifest saved: {out_csv}")
    print(f"   total={len(df)}, trainable={int(df['trainable'].sum())}, date_range={df['date'].min()}..{df['date'].max()}")
    quick_summary(df, f"{data_type} manifest")
    return df

def save_splits(df: pd.DataFrame, data_type: str):
    splits = {
        'train': df[(df['trainable'] == True) & (df['year'].isin(TRAIN_YRS))],
        'val':   df[(df['trainable'] == True) & (df['year'].isin(VAL_YRS))],
        'test':  df[(df['trainable'] == True) & (df['year'].isin(TEST_YRS))],
    }
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    for name, sdf in splits.items():
        quick_summary(sdf, f"{data_type} {name}")
        if sdf.empty:
            continue
        txt = os.path.join(OUT_DIR, f"{data_type.lower()}_{name}_{ts}.txt")
        csv = os.path.join(OUT_DIR, f"{data_type.lower()}_{name}_{ts}.csv")
        with open(txt, "w") as f:
            for d in sdf['date'].tolist():
                f.write(f"{d}\n")
        sdf[['date','path','coverage','season','coverage_source','threshold','qc_pass']].to_csv(csv, index=False)
        print(f"   ✅ {name}: {len(sdf)} -> {txt} + {csv}")

# ----------------------------- Run -----------------------------
print("🚀 Building unified manifests and splits with QC guards...")

so2_df = build_manifest("SO2", BASE_SO2)
save_splits(so2_df, "SO2")

no2_df = build_manifest("NO2", BASE_NO2)
save_splits(no2_df, "NO2")

print("\n🎉 Done.")

🚀 Building unified manifests and splits with QC guards...

📋 Building manifest for SO2 ...
  2019: 365 files
  2020: 366 files
  2021: 365 files
  2022: 365 files
  2023: 365 files
✅ SO2 manifest saved: /content/drive/MyDrive/Variables/so2_manifest.csv
   total=1826, trainable=852, date_range=2019-01-01..2023-12-31

[SO2 manifest] files=1826  cov_mean=0.107  cov_med=0.086
 season counts:
 season
spring    460
summer    460
autumn    455
winter    451
Name: count, dtype: int64
 coverage_source breakdown:
 coverage_source
domain+data    1826
Name: count, dtype: int64

[SO2 train] files=507  cov_mean=0.199  cov_med=0.188
 season counts:
 season
summer    224
spring    168
autumn     93
winter     22
Name: count, dtype: int64
 coverage_source breakdown:
 coverage_source
domain+data    507
Name: count, dtype: int64
   ✅ train: 507 -> /content/drive/MyDrive/Variables/so2_train_20250912_093025.txt + /content/drive/MyDrive/Variables/so2_train_20250912_093025.csv

[SO2 val] files=175  cov_mean=

In [ ]:
import os, json, numpy as np, pandas as pd

OUT = "/content/drive/MyDrive/Variables"

def freeze_scaler(path, name):
    with np.load(path, allow_pickle=True) as d:
        scalers = d['scalers'].item()
        feats = [k for k in d['feature_order'].tolist()]
        meta = d['metadata'].item() if 'metadata' in d.files else {}
    info = {
        "file": path, "num_features": len(feats), "features": feats,
        "scaler_keys": sorted([k for k,v in scalers.items() if 'mean' in v]),
        "metadata": meta
    }
    with open(os.path.join(OUT, f"{name}_scaler_frozen.json"), "w") as f:
        json.dump(info, f, indent=2)
    print(f"✅ freeze: {name} -> {os.path.join(OUT, f'{name}_scaler_frozen.json')}")

def split_summary(csv_path, tag):
    df = pd.read_csv(csv_path)
    summ = {
        "files": len(df),
        "cov_mean": float(df['coverage'].mean()),
        "cov_median": float(df['coverage'].median()),
        "season_counts": df['season'].value_counts().to_dict(),
        "source_counts": (df['coverage_source'].value_counts().to_dict()
                          if 'coverage_source' in df.columns else {})
    }
    out = csv_path.replace(".csv", "_summary.json")
    with open(out, "w") as f: json.dump(summ, f, indent=2)
    print(f"✅ summary: {tag} -> {out}")

# 1) 冻结两份scaler
freeze_scaler("/content/drive/MyDrive/no2_scalers_2019_2021.npz", "no2")
freeze_scaler("/content/drive/MyDrive/so2_scalers_2019_2021.npz", "so2")

# 2) 为每个split保存摘要（NO2/SO2 各自 train/val/test）
for p in [
    "so2_train", "so2_val", "so2_test",
    "no2_train", "no2_val", "no2_test"
]:
    # 若文件名带时间戳，可用最新一个
    from glob import glob
    cand = sorted(glob(os.path.join(OUT, f"{p}_*.csv")))
    if cand:
        split_summary(cand[-1], p)

✅ freeze: no2 -> /content/drive/MyDrive/Variables/no2_scaler_frozen.json
✅ freeze: so2 -> /content/drive/MyDrive/Variables/so2_scaler_frozen.json
✅ summary: so2_train -> /content/drive/MyDrive/Variables/so2_train_20250912_093025_summary.json
✅ summary: so2_val -> /content/drive/MyDrive/Variables/so2_val_20250912_093025_summary.json
✅ summary: so2_test -> /content/drive/MyDrive/Variables/so2_test_20250912_093025_summary.json
✅ summary: no2_train -> /content/drive/MyDrive/Variables/no2_train_20250912_095739_summary.json
✅ summary: no2_val -> /content/drive/MyDrive/Variables/no2_val_20250912_095739_summary.json
✅ summary: no2_test -> /content/drive/MyDrive/Variables/no2_test_20250912_095739_summary.json
